<p> <center> <a href="../../Start_Here.ipynb">Home Page</a> </center> </p>

 
<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="Multitask_Prompt_and_PTuning.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 33%; text-align: center;">
       
    
</div>

# Lab Activity 2

## Instructions

In this lab, you are to reproduce the p-tuning and prompt-tuning process for the question-answering task using the CUAD dataset already available. To complete the activity, you are to implement the following steps:

- Understand the structure of the CUAD dataset and process the `CUAD_v1.json` file into a training and validation set. You are free to decide the split ratio. More detail is given below.
- Setup your P-Tuning Model Config using omegaconf
- Create the prompt formatting template and set the task
- Set the pre-trained GPT model as `nemo_gpt1.3B_fp16.nemo` 
- Build the PyTorch lightning trainer and set all hyperparameters
- Create the NeMo experiment manager
- Run your p-tuning session
- Restore your p-tuned model to run inference


Part of the code is written for you. You are to complete the rest by filling in the statements with the missing value(s) in the commented areas of the notebook. We recommend consciously setting/modifying the `config.trainer.max_epochs` value as it determines the time to complete the lab. The lab activity should not exceed `45 mins`; therefore, the value of `config.trainer.max_epochs` should be between `1 and 6` as one epoch may take up to `7 mins`. If you plan to exceed these values, run this notebook after the Bootcamp active hours.

Note: *You are not expected to get the best result as this activity is for learning. To achieve better results, you can modify parameters: the number of epochs, learning rate, batch size, max step, training, and validation set sample size*.

<div style="text-align:left; color:#FF0000; height:40px; text-color:red; font-size:20px">Before you run this notebook, please close and shut down the kernel of the previous notebooks. </div>

In [ ]:
import os
import wget

# Tasks and Datasets
We will be using p-tuning to teach our GPT model to do **Question Answering**.

We will be using the [Contract Understanding Atticus Dataset (CUAD) v1](https://www.atticusprojectai.org/cuad). It is a corpus of more than 13,000 labels in 510 commercial legal contracts that have been manually labeled to identify 41 categories of essential clauses that lawyers look for when reviewing contracts in connection with corporate transactions. 

CUAD is curated and maintained by The Atticus Project, Inc. to support NLP research and development in legal contract review. Analysis of CUAD can be found at https://arxiv.org/abs/2103.06268. Code for replicating the results and the trained model can be found at https://github.com/TheAtticusProject/cuad.

To download the CUAD dataset, you can visit the [CUAD v1 website](https://www.atticusprojectai.org/cuad).



## CUAD Dataset Structure
### Data Instances


```json
...
{  
    "answers": {
        "answer_start": [44],
        "text": ["DISTRIBUTOR AGREEMENT"]
    },
    "context": "EXHIBIT 10.6\n\n DISTRIBUTOR AGREEMENT\n\n THIS  DISTRIBUTOR  AGREEMENT (the  \"Agreement\")  is made    by and between Electric City Corp.,  a Delaware  corporation  (\"Company\")  and Electric City of Illinois LLC (\"Distributor\") this 7th day of September, 1999...",
    "id": "LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT__Document Name_0",
    "question": "Highlight the parts (if any) of this contract related to \"Document Name\" that should be reviewed by a lawyer. Details: The name of the contract",
    "title": "LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT"
}
...
```

### Data Fields

- **id:** a string feature representing a unique question ID.
- **title:** represents domain/topic of discussion or document title.
- **context**: represents a group sentences or document where the answer(s) to a question(s) lies. It is possible for a single context to have two or more questions.
- **question:** a text that requires an answer from a context.
- **answers:** a text list containing a dictionary of responses and the index `(answer_start)` where the answer text starts within a context.

More information on CUAD dataset can be found on the [CUAD v1 website](https://www.atticusprojectai.org/cuad) or at the [Hugging Face Datasets page](https://huggingface.co/datasets/cuad).


## File Formats

The files in CUAD v1 include 1 CSV file, 1 SQuAD-style JSON file, 28 Excel files, 510 PDF files, and 510 TXT files.

-  1 master clauses CSV: an 83-column 511-row file. The first column contains the contracts' names corresponding to the PDF and TXT files in the "full_contracts_pdf" and "full_contracts_txt" folders. The remaining columns have (1) text context (sometimes referred to as clause) and (2) human-input answers that correspond to each of the 41 categories in these contracts. 

- 1 SQuAD-style JSON: this file is derived from the master clauses CSV to follow the same format as SQuAD 2.0 (https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/), a question answering dataset whose answers are similarly spans of the input text. The exact structure of the JSON format exactly mimics that of SQuAD 2.0 for compatibility with prior work. We also provide Python scripts for processing this data for ease of use.

- 28 Excels: a collection of Excel files containing clauses responsive to each category. The first column contains the contracts' names corresponding to the PDF and TXT files in the "full_contracts_pdf" and "full_contracts_txt" folders. The remaining columns have (1) text context (clause) corresponding to one or more Categories that belong in the same group as identified in the "Category List" below, and (2), in some cases, human-input answers that correspond to such text context. Each file is named "Label Report - [label/group name] (Group [number]).xlsx"

- 510 entire contract PDFs: a collection of the underlying contracts we used to extract the labels. Each file is named "[document name].pdf". These contracts are in a PDF format and are not labeled. The entire agreement PDFs contain raw data and are provided for context and reference.

- 510 full contract TXTs: a collection of TXT files of the underlying contracts. Each file is named "[document name].txt". These contracts are in a plaintext format and are not labeled. The full contract TXTs contain raw data and are provided for context and reference.

We will use the SQuAD-style JSON format dataset (CUAD_v1.json) to facilitate work with prior work and existing language models. In this dataset, each contract is broken up into paragraphs. For each provision category, a model must predict the span of text (if any) in that paragraph corresponding to that category.

**Extract from SQuAD-style JSON format of CUAD dataset**

```json
{
  "qas": [
    {
      "answers": [{ "text": "Promotion and Distribution Agreement","answer_start": 307 }],
      "id": "WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION AND DISTRIBUTION AGREEMENT__Document Name",
      "question": "Highlight the parts (if any) of this contract related to \"Document Name\" that should be reviewed by a lawyer. Details: The name of the contract",
      "is_impossible": false
    },
    {
      "answers": [{"text": "Distributor","answer_start": 625 },
                  {"text": "Google","answer_start": 644 },
                  {"text": "Google Inc","answer_start": 644 },
                  {"text": "Whitesmoke Inc.","answer_start": 492 }],
      "id": "WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION AND DISTRIBUTION AGREEMENT__Parties",
      "question": "Highlight the parts (if any) of this contract related to \"Parties\" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract",
      "is_impossible": false
    },
    ...............
    {
      "answers": [],
      "id": "WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION AND DISTRIBUTION AGREEMENT__Agreement Date",
      "question": "Highlight the parts (if any) of this contract related to \"Agreement Date\" that should be reviewed by a lawyer. Details: The date of the contract",
      "is_impossible": true
    }
  ],
  "context": "Exhibit 10.26    CONFIDENTIAL TREATMENT HAS BEEN REQUESTED AS TO CERTAIN PORTIONS OF THIS DOCUMENT.      EACH SUCH PORTION,  WHICH HAS BEEN OMITTED HEREIN AND REPLACED WITH AN ASTERISK [*], HAS BEEN FILED SEPARATELY      WITH THE SECURITIES AND EXCHANGE COMMISSION.     PROMOTION AND DISTRIBUTION AGREEMENT     This Promotion and        Distribution Agreement including all exhibits (collectively referred to as the \"Agreement\"), effective as of 1    August 2011 (the  \"Effective Date\"), is made by and between Whitesmoke Inc., with registered offices/principle    place of business at 501 Silverside Road, Suite 105,  Wilmington DE 19809, USA, (\"Distributor\"),......"
}


## Data Preparation

### 1. Dataset splitting: Training and Testing Samples

- A CuAD dataset in the SQuAD-like JSON format is provided as a single dataset file.
- You must split the dataset into separate training and testing subsets to facilitate model training and evaluation.
- You can achieve this using the following code snippet, demonstrating the dataset-splitting process.

Run the cell below to download the CUAD dataset.

In [1]:
!python3 ../../source_code/dataset_cuad.py

Downloading...
From: https://drive.google.com/uc?id=1C5T6Na63oATxn8rBmZ5YT55mjrs7WTEG&confirm=t
To: /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/data/CUAD_v1.json
100%|██████████████████████████████████████| 40.1M/40.1M [00:00<00:00, 63.2MB/s]


- Before running the code snippet for dataset splitting, please ensure that the `CUAD_v1.json` dataset file exists in the `/workspace/data` project directory.

In [4]:
import json
import random
import os

def split_dataset(data, train_ratio):
    # Calculate the number of training samples
    num_train = int(len(data) * train_ratio)

    # Shuffle the data
    random.shuffle(data)

    # Split the data into training and testing sets
    train_data = data[:num_train]
    test_data = data[num_train:]

    return train_data, test_data

# Set the working directory
data_directory = '../../data/'

# Define the path to the downloaded CUAD_v1.json dataset 
# Make sure that the CUAD_v1.json dataset is in the same directory as the script or provide the correct path to it in the cuad_file_path variable.)
cuad_file_path = os.path.join(data_directory, 'CUAD_v1.json')

# Load the JSON file
with open(cuad_file_path) as f:
    json_data = json.load(f)

# Extract the data from the JSON file
data = json_data['data']

################### You may modify to the train ratio of your preference ############################

# Split the dataset into train and test sets
train_data, test_data = split_dataset(data, train_ratio=0.9)

#####################################################################################################

# Create the 'data' directory if it doesn't exist
cuad_directory = os.path.join(data_directory, 'cuad')
if not os.path.exists(cuad_directory):
    os.makedirs(cuad_directory)

# Save the train data into a separate JSON file
train_file_path = os.path.join(cuad_directory, 'train_dataset.json')
with open(train_file_path, 'w') as train_file:
    json.dump({'version': json_data['version'], 'data': train_data}, train_file)

# Save the test data into a separate JSON file
test_file_path = os.path.join(cuad_directory, 'test_dataset.json')
with open(test_file_path, 'w') as test_file:
    json.dump({'version': json_data['version'], 'data': test_data}, test_file)


### 2. Preprocessing the train and test datasets

The prompt learning dataset loader accepts a list of JSON/dictionary objects or a list of JSON file names where each file contains a collection of JSON objects. Each JSON object must include the field `taskname,` a string identifier for the task the data example corresponds to. They should also have one or more fields corresponding to different sections of the discrete text prompt. The input data might look like:

```
[
    {"taskname": "cuad", "context": [CONTEXT_PARAGRAPH_TEXT1], "question": [QUESTION_TEXT1], "answer": [ANSWER_TEXT1]},
    {"taskname": "cuad", "context": [CONTEXT_PARAGRAPH_TEXT2], "question": [QUESTION_TEXT2], "answer": [ANSWER_TEXT2]},
]
```

These additional fields can be unlimited and will be used to help map different parts of the discrete text input to a prompt template you define.

Set the working directory as `WORK_DIR` and the data directory as `CUAD_DIR.` 

In [5]:
WORK_DIR = '../../'
CUAD_DIR = "../../data/cuad"

Make a directory to store Python scripts for multitask p-tuning

In [6]:
SCRIPT_DIR = os.path.join(WORK_DIR, "source_code/activity2")

Preprocessing the dataset using the `prompt_learning_cuad_preprocessing.py` script located in the `source_code` directory


In [7]:
# Preprocess cuad data
!python $WORK_DIR/source_code/prompt_learning_cuad_preprocessing.py --data-dir {CUAD_DIR}

Saving train split to ../../data/cuad/cuad_train.jsonl
100%|█████████████████████████████████████| 6071/6071 [00:01<00:00, 4386.72it/s]
Saving val split to ../../data/cuad/cuad_val.jsonl
100%|███████████████████████████████████████| 631/631 [00:00<00:00, 4244.35it/s]
Saving test split to ../../data/cuad/cuad_test_ground_truth.jsonl
100%|█████████████████████████████████████| 2091/2091 [00:00<00:00, 5737.65it/s]
Saving test split to ../../data/cuad/cuad_test.jsonl
100%|█████████████████████████████████████| 2091/2091 [00:00<00:00, 5765.94it/s]


In [8]:
# What the cuad dataset looks like after processing
!head -1 $CUAD_DIR/cuad_train.jsonl

{"taskname": "cuad", "context": "Exhibit 10.11\n\nDATED 26 September 2019\n\nBicycleTX Ltd\n\nand\n\nNigel Crockett\n\n___________________________________________________\n\nSERVICE AGREEMENT\n\n___________________________________________________\n\n\n\n\n\nTHIS AGREEMENT is made on 26 September 2019\n\nBETWEEN:\n\n(1) BICYCLETX LIMITED a company incorporated under the laws of England and Wales (Company Number 11036101) whose registered office is at Building 900 Babraham Research Campus, Babraham, Cambridgeshire, CB22 3AT, United Kingdom (the \"Company\"); and\n\n(2) NIGEL CROCKETT of (the ''Employee\").\n\nIT IS AGREED as follows:\n\n1. COMMENCEMENT OF EMPLOYMENT\n\n1.1 This Agreement shall take effect 26  September 2019 (the \"Effective Date\").\n\n1.2 Your employment shall commence on 26  September 2019 and shall continue unless and until either party gives notice to the other in accordance with paragraph 11 below. No employment with a previous employer is deemed to be continuous wi

##### Expected Output
```python
{"taskname": "cuad", "context": "Exhibit 10.2 PORTIONS OF THIS EXHIBIT MARKED BY [**] HAVE BEEN OMITTED PURSUANT TO RULE 601(B)(10) OF REGULATION S-K. THE OMITTED INFORMATION IS (I) NOT MATERIAL AND (II) WOULD LIKELY CAUSE COMPETITIVE HARM TO THE REGISTRANT IF PUBLICLY DISCLOSED. EXECUTION VERSION STRATEGIC ALLIANCE AGREEMENT STRATEGIC ALLIANCE AGREEMENT, dated as of December 20, 2019 (as amended, supplemented or otherwise modified from time to time, this \"Agreement\"), by and among Farids & Co. LLC, a Delaware limited liability company (\"Farids\"), Edible Arrangements, LLC, a Delaware limited liability company (\"EA\"), and Rocky Mountain Chocolate Factory, Inc., a Delaware corporation (the \"Company\"). W I T N E S S E T H: WHEREAS, the Company is an international franchisor, confectionery manufacturer and retail operator; WHEREAS, Farids is a holding company and, together with TF (as defined below), indirectly controls EA; WHEREAS, EA is a US-based franchisor that specializes in fresh fruit arrangements and specialty fruit gift items; WHEREAS, the Company desires to issue and sell,...If the foregoing applies, the Parties shall use all reasonable endeavours to agree within a reasonable time upon any lawful and reasonable  variations to the     18\n\n\n\n\n\n     Agreement which may be necessary in order to achieve, to the greatest extent possible, the same effect as would have been achieved by  the Clause, or the part of the Clause, in question.     22 GOVERNING LAW     22.1 This Agreement is governed by English law.     22.2 The Parties submit to the non-exclusive jurisdiction of the courts of England and Wales.     This Agreement shall come into force on the date given at the beginning of this Agreement.\n\n   19\n\nSIGNED by\n\n   )        )  (name),                     )   a duly authorised signatory of     ) (signature)  SHBV (HONG KONG) LTD            )\n\nSIGNED by\n\n      )           )  (name),\n\n\n\n\n\n     )\n\na duly authorised signatory of     ) (signature)  WASTE2ENERGY GROUP HOLDINGS PLC      )", "question": "Highlight the parts (if any) of this contract related to \"Document Name\" that should be reviewed by a lawyer. Details: The name of the contract","answer": "STRATEGIC ALLIANCE AGREEMENT, d"}
```

We made a `.jsonl` file for each train, validation, and testing split of the cuad data. Every `.jsonl` file contains JSON objects with the fields `taskname,` `context,` `question,` and `answer.` The preprocessing script is called `prompt_learning_cuad_preprocessing.py.`
The CUAD dataset comprises a collection of contract-related questions and answers. It consists of various contracts like "LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT", "WHITESMOKE, INC_11_08_2011-EX-10.26-PROMOTION AND DISTRIBUTION AGREEMENT", and "MetLife, Inc. - Remarketing Agreement." Each contract has a title and paragraph associated with it, and each paragraph has several related questions and answers. When we separated the train/validation/test splits, we separated them on the title level. For example, if the training set contains paragraphs and questions about the contract `LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGREEMENT`, neither the validation nor test samples will have any questions on this contract. All questions about a specific contract are isolated to one data split.

 (Like the Financial PhraseBank Dataset, we randomly selected 80% of the questions for training, 10% for validation, and 10% for testing. The process resulted in `69125` test examples, `8952` validation, and `8744` testing examples. The `answer` field was removed from test examples.)

Training on the whole train split could take a lot of time, so we will clip the train set to 1.5k examples for this lab activity and limit the validation dataset to 150 samples.

In [9]:
! head -1500 $CUAD_DIR/cuad_train.jsonl > $CUAD_DIR/cuad_short_train.jsonl
! head -150 $CUAD_DIR/cuad_val.jsonl > $CUAD_DIR/cuad_short_val.jsonl

## P-Tuning Model Config Setup

In [10]:
from omegaconf import OmegaConf

# Load the example config file so we can start editing it
CONFIG_PATH = os.path.join(WORK_DIR, "source_code/activity2/conf/megatron_gpt_prompt_learning_config.yaml")
config = OmegaConf.load(CONFIG_PATH)

Set the train and validation dataset in the config

In [11]:
config.model.data.train_ds = [f"{CUAD_DIR}/cuad_short_train.jsonl"]
config.model.data.validation_ds = [f"{CUAD_DIR}/cuad_short_val.jsonl"]


### Prompt Formatting

- Add value to:  
```bash
      "taskname": "",
     "prompt_template": "",
     ...
```

In [12]:
config.model.task_templates = [
    {
        "taskname": "cuad",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> Context: {context}\n\nQuestion: {question}\n\nAnswer:{answer}",
        "total_virtual_tokens": 15,
        "virtual_token_splits": [15],
        "truncate_field": "context",
        "answer_only_loss": True,
        "answer_field": "answer",
    },
]

### Setting New Tasks

You do this by setting the config file's `new_tasks` and `existing_tasks` values. Because we are p-tuning a model with no existing tasks, you should set `existing_tasks=[]` and `new_tasks=["cuad"]` as follows:

In [13]:
config.model.existing_tasks = []
config.model.new_tasks = ["cuad"]

### Set The Pre-Trained GPT Model

In [14]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
MegatronGPTModel.list_available_models()

[PretrainedModelInfo(
 	pretrained_model_name=megatron_gpt_345m,
 	description=345M parameter GPT generative Megatron model.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
 )]

Download the `nemo_gpt1.3B_fp16.nemo` model

In [15]:
gpt_file_name = "nemo_gpt1.3B_fp16.nemo"
PRETRAINED_MODEL_DIR = os.path.join(WORK_DIR, "source_code")

# download the nemo_gpt1.3B_fp16.nemo
!python3 ../../source_code/megatron-gpt-model.py

Downloading...
From: https://drive.google.com/uc?id=1mTQjczmQQTm-TL0lxxfzoq3EEFoKQ2l6&confirm=t
To: /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/source_code/nemo_gpt1.3B_fp16.nemo
100%|██████████████████████████████████████| 3.02G/3.02G [00:34<00:00, 86.4MB/s]


- Now that you have a `.nemo` GPT file, please add its path to the prompt learning config.

In [16]:
# Set GPT model path on prompt learning config
config.model.language_model_path = f'{PRETRAINED_MODEL_DIR}/nemo_gpt1.3B_fp16.nemo'

- Set checkpoint behavior

In [17]:
config.exp_manager.checkpoint_callback_params.save_nemo_on_train_end= True
config.exp_manager.checkpoint_callback_params.always_save_nemo= True
config.exp_manager.checkpoint_callback_params.save_best_model= True

### Setting P-Tuning Specific Params

In [18]:
# Set the virtual prompt style to p-tuning (already set by default)
config.model.virtual_prompt_style = "p-tuning"


config.model.p_tuning.dropout = 0.0
config.model.p_tuning.num_layers = 2
config.model.global_batch_size = 2
config.model.micro_batch_size = 1
config.model.max_seq_length = 10240  # you can adjust to the squence length

Let's have a look at all the values you've set in the model config.

In [19]:
# Final model config
print(OmegaConf.to_yaml(config.model))

seed: 1234
nemo_path: ${name}.nemo
virtual_prompt_style: p-tuning
tensor_model_parallel_size: 1
pipeline_model_parallel_size: 1
global_batch_size: 2
micro_batch_size: 1
validation_global_batch_size: ${model.global_batch_size}
validation_micro_batch_size: ${model.micro_batch_size}
validation_drop_last: false
restore_path: null
language_model_path: ../../source_code/nemo_gpt1.3B_fp16.nemo
save_nemo_on_validation_end: true
existing_tasks: []
new_tasks:
- cuad
sequence_parallel: false
activations_checkpoint_granularity: null
activations_checkpoint_method: null
activations_checkpoint_num_layers: null
task_templates:
- taskname: cuad
  prompt_template: '<|VIRTUAL_PROMPT_0|> Context: {context}


    Question: {question}


    Answer:{answer}'
  total_virtual_tokens: 15
  virtual_token_splits:
  - 15
  truncate_field: context
  answer_only_loss: true
  answer_field: answer
prompt_tuning:
  new_prompt_init_methods:
  - text
  new_prompt_init_text:
  - some init text goes here
p_tuning:
  encode

## Building the PyTorch Lightning Trainer

*Tips:* 
- *Check the Multitask Prompt-tuning and P-tuning notebook*
- *Set the `config.trainer.max_epochs` to values from 1 to 5 (each epoch takes approximately 7mins running on an A100 MIG instance (32GB))*

In [20]:
import torch
import pytorch_lightning as pl
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPStrategy
from pytorch_lightning.plugins.environments import TorchElasticEnvironment

# let's modify some trainer configs
# check if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.accelerator = accelerator
config.trainer.devices = 1

####### Set from 5 to 10 epochs. Each epoch takes approximately 7mins running on A100 MIG instance(32GB) ########
config.trainer.max_epochs = 5  
##########################################################################################################
config.trainer.val_check_interval = 1.0

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# setup cluster environment parameters"
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'

strategy = NLPDDPStrategy(find_unused_parameters=False,no_ddp_communication_hook=True)
plugins = [TorchElasticEnvironment()]
trainer = pl.Trainer(plugins= plugins, strategy=strategy, **config.trainer)

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


Trainer config - 

devices: 1
accelerator: gpu
num_nodes: 1
precision: 16
logger: false
enable_checkpointing: false
replace_sampler_ddp: false
max_epochs: 5
max_steps: -1
log_every_n_steps: 10
val_check_interval: 1.0
gradient_clip_val: 1.0
resume_from_checkpoint: null
benchmark: false



## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [21]:
from nemo.utils.exp_manager import exp_manager

# Set name of the experiment
config.name = 'p_tuning'
config.exp_manager.resume_if_exists = False
# Init the experiment manager and view the exp_dir
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)
print(exp_dir)

[NeMo I 2024-03-06 10:41:17 exp_manager:374] Experiments will be logged at /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17
[NeMo I 2024-03-06 10:41:17 exp_manager:797] TensorboardLogger has been set up
/net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17


Set the learning rate and the precision values.

In [22]:
# Set some of the learning parameters
config.model.optim.lr = 1e-4
config.model.precision = config.trainer.precision

## P-Tuning Session

The only thing left is to load up the model and begin p-tuning!

In [23]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_prompt_learning_model import MegatronGPTPromptLearningModel

model = MegatronGPTPromptLearningModel(cfg=config.model,trainer=trainer)

[NeMo I 2024-03-06 10:41:35 megatron_init:234] Rank 0 has data parallel group: [0]
[NeMo I 2024-03-06 10:41:35 megatron_init:237] All data parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:41:35 megatron_init:238] Ranks 0 has data parallel rank: 0
[NeMo I 2024-03-06 10:41:35 megatron_init:246] Rank 0 has model parallel group: [0]
[NeMo I 2024-03-06 10:41:35 megatron_init:247] All model parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:41:35 megatron_init:257] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-03-06 10:41:35 megatron_init:261] All tensor model parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:41:35 megatron_init:262] Rank 0 has tensor model parallel rank: 0
[NeMo I 2024-03-06 10:41:35 megatron_init:276] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2024-03-06 10:41:35 megatron_init:288] Rank 0 has embedding group: [0]
[NeMo I 2024-03-06 10:41:35 megatron_init:294] All pipeline model parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:41:35 megatron_init:295]

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2024-03-06 10:42:17 megatron_base_model:264] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2024-03-06 10:42:20 nlp_overrides:401] Model MegatronGPTModel was successfully restored from /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/source_code/nemo_gpt1.3B_fp16.nemo.
[NeMo I 2024-03-06 10:42:20 auto_tokenizer:172] 15 special tokens added, resize your model accordingly.


Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2024-03-06 10:43:00 megatron_init:234] Rank 0 has data parallel group: [0]
[NeMo I 2024-03-06 10:43:00 megatron_init:237] All data parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:43:00 megatron_init:238] Ranks 0 has data parallel rank: 0
[NeMo I 2024-03-06 10:43:00 megatron_init:246] Rank 0 has model parallel group: [0]
[NeMo I 2024-03-06 10:43:00 megatron_init:247] All model parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:43:00 megatron_init:257] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-03-06 10:43:00 megatron_init:261] All tensor model parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:43:00 megatron_init:262] Rank 0 has tensor model parallel rank: 0
[NeMo I 2024-03-06 10:43:00 megatron_init:276] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2024-03-06 10:43:00 megatron_init:288] Rank 0 has embedding group: [0]
[NeMo I 2024-03-06 10:43:00 megatron_init:294] All pipeline model parallel group ranks: [[0]]
[NeMo I 2024-03-06 10:43:00 megatron_init:295]

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2024-03-06 10:43:00 megatron_base_model:264] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2024-03-06 10:43:03 nlp_overrides:401] Model MegatronGPTModel was successfully restored from /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/source_code/nemo_gpt1.3B_fp16.nemo.
[NeMo I 2024-03-06 10:43:03 auto_tokenizer:172] 15 special tokens added, resize your model accordingly.


Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


Note: Each training epoch takes around 7 mins but may vary based on the device used.

In [25]:
os.environ["MASTER_PORT"] = "51234"

trainer.fit(model)

Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------



[NeMo I 2024-03-06 10:45:52 gpt_prompt_learning_dataset:85] Loading and tokenizing dataset ... 


0it [00:00, ?it/s]

[NeMo I 2024-03-06 10:45:52 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 10:45:52 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 10:45:53 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 10:45:53 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 10:45:53 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 10:45:53 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 10:45:53 gpt_prompt_learning_dataset:274] Input greater than max sequence l

0it [00:00, ?it/s]

...

Epoch 4, global step 3750: 'val_loss' reached 1.57781 (best 1.57710), saving model to '/net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/megatron_gpt_prompt_tune--val_loss=1.578-step=3750.ckpt' as top 2


[NeMo I 2024-03-06 10:57:54 nemo_model_checkpoint:168] New best .nemo model saved to: /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/p_tuning.nemo
[NeMo I 2024-03-06 10:57:54 nlp_overrides:231] Removing checkpoint: /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/megatron_gpt_prompt_tune--val_loss=1.584-step=2250.ckpt
[NeMo I 2024-03-06 10:57:54 nemo_model_checkpoint:168] New best .nemo model saved to: /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/p_tuning.nemo
[NeMo I 2024-03-06 10:57:54 nlp_overrides:231] Removing checkpoint: /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/megatron_gpt_prompt_tune--val_loss=1

`Trainer.fit` stopped: `max_epochs=5` reached.


[NeMo I 2024-03-06 10:57:54 nemo_model_checkpoint:168] New best .nemo model saved to: /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/p_tuning.nemo


Restoring states from the checkpoint path at /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/megatron_gpt_prompt_tune--val_loss=1.577-step=3000.ckpt
Restored all states from the checkpoint file at /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/megatron_gpt_prompt_tune--val_loss=1.577-step=3000.ckpt


## Inference After P-Tuning

- Restore the Megatron-GPT Prompt Learning Model from the specified checkpoint path by `trainer.fit(model)` output above `eg:'../../jupyter_notebook/nemo/nemo_experiments/p_tuning/2023-08-17_02-38-32/checkpoints/p_tuning.nemo'`

In [28]:
# modify the restore_path to the path specified in the trainer.fit(model) output
import nemo.collections.nlp as nemo_nlp

mymodel = nemo_nlp.models.language_modeling.megatron_gpt_prompt_learning_model.MegatronGPTPromptLearningModel.restore_from(restore_path='/net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/p_tuning.nemo',trainer=trainer)
# You can perform inference using this restored model

[NeMo I 2024-03-06 11:00:30 megatron_init:234] Rank 0 has data parallel group: [0]
[NeMo I 2024-03-06 11:00:30 megatron_init:237] All data parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:00:30 megatron_init:238] Ranks 0 has data parallel rank: 0
[NeMo I 2024-03-06 11:00:30 megatron_init:246] Rank 0 has model parallel group: [0]
[NeMo I 2024-03-06 11:00:30 megatron_init:247] All model parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:00:30 megatron_init:257] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-03-06 11:00:30 megatron_init:261] All tensor model parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:00:30 megatron_init:262] Rank 0 has tensor model parallel rank: 0
[NeMo I 2024-03-06 11:00:30 megatron_init:276] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2024-03-06 11:00:30 megatron_init:288] Rank 0 has embedding group: [0]
[NeMo I 2024-03-06 11:00:30 megatron_init:294] All pipeline model parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:00:30 megatron_init:295]

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2024-03-06 11:01:11 megatron_base_model:264] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2024-03-06 11:01:11 build_model:143]  > number of parameters on (tensor, pipeline) model parallel rank (0, 0): 812380160
[NeMo I 2024-03-06 11:01:14 nlp_overrides:401] Model MegatronGPTModel was successfully restored from /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/source_code/nemo_gpt1.3B_fp16.nemo.
[NeMo I 2024-03-06 11:01:14 auto_tokenizer:172] 15 special tokens added, resize your model accordingly.


Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2024-03-06 11:01:54 megatron_init:234] Rank 0 has data parallel group: [0]
[NeMo I 2024-03-06 11:01:54 megatron_init:237] All data parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:01:54 megatron_init:238] Ranks 0 has data parallel rank: 0
[NeMo I 2024-03-06 11:01:54 megatron_init:246] Rank 0 has model parallel group: [0]
[NeMo I 2024-03-06 11:01:54 megatron_init:247] All model parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:01:54 megatron_init:257] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-03-06 11:01:54 megatron_init:261] All tensor model parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:01:54 megatron_init:262] Rank 0 has tensor model parallel rank: 0
[NeMo I 2024-03-06 11:01:54 megatron_init:276] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2024-03-06 11:01:54 megatron_init:288] Rank 0 has embedding group: [0]
[NeMo I 2024-03-06 11:01:54 megatron_init:294] All pipeline model parallel group ranks: [[0]]
[NeMo I 2024-03-06 11:01:54 megatron_init:295]

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2024-03-06 11:01:54 megatron_base_model:264] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2024-03-06 11:01:54 build_model:143]  > number of parameters on (tensor, pipeline) model parallel rank (0, 0): 812380160
[NeMo I 2024-03-06 11:01:56 nlp_overrides:401] Model MegatronGPTModel was successfully restored from /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/source_code/nemo_gpt1.3B_fp16.nemo.
[NeMo I 2024-03-06 11:01:56 auto_tokenizer:172] 15 special tokens added, resize your model accordingly.


Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2024-03-06 11:01:56 save_restore_connector:249] Model MegatronGPTPromptLearningModel was successfully restored from /net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/p_tuning.nemo.


### Run the inference using the test examples below

In [29]:
test_examples = [
    {'taskname': 'cuad',
  'context': 'Exhibit 10.41\n\nSPONSORSHIP AGREEMENT\n\nThis Sponsorship Agreement (the "Agreement") is entered into effective January 1, 2010 by and between Stallings Capital Group  Consultants, Ltd., a Texas limited partnership dba Bob Stallings Racing ("Racing"), and GAINSCO, INC., a Texas corporation (the "Sponsor").\n\nRacing organized and operated a racing team engaging in Daytona Prototype Series auto racing (the "Racing Team") in professional races in  2005 through 2008, and the Sponsor was the primary sponsor of the Racing Team pursuant to Sponsorship Agreements dated February 7, 2005,  February 1, 2006, January 1, 2007, January 1, 2008 and January 1, 2009. Racing has invited the Sponsor to continue to act as the primary sponsor of  the Racing Team for 2010, and the Sponsor desires to act in that capacity. In consideration of the sponsorship fee provided for herein, the parties  desire to enter into this Agreement to govern the terms of such sponsorship in 2010.\n\nNow, therefore, Racing and the Sponsor hereby agree as follows:\n\n1. Term. Subject to the provisions of Section 14 hereof, the term of this Agreement and the sponsorship described herein shall commence on  January 1, 2010 and extend through December 31, 2010.\n\n2. Advertising and Other Benefits. Subject to payment by the Sponsor of the sponsorship fee provided for herein, during the term of this  Agreement Racing shall cause the Racing Team to provide for the Sponsor\'s benefit all of the benefits customarily associated with the  sponsorship of a Daytona Prototype Series racing team and consistent with the benefits provided to the Sponsor in 2005 - 2009 (individually, a  "Benefit," and collectively, the "Benefits"), including but not limited to the following:   (i) displaying prominent identification of the Sponsor\'s name and/or logo in signage on the race car and racing suits and, where  appropriate, on other team equipment (subject to approval by the Sponsor);   (ii) making available for the use of the Sponsor (x) the personalities associated with the Racing Team, including without limitation  the name, voice, picture, portrait, likeness, persona and/or signature of each driver for endorsements, commercial advertising and  promotions in any and all media throughout the world during the term of this Agreement, (y) the Racing Team\'s home base facilities in  Texas, and (z) those facilities designated or assigned for the use of the Racing Team at each race and race location at which the Racing  Team actually participates in the race, all for appropriate public relations and other promotional and marketing purposes. Racing agrees  that it will actively participate in the Rolex 24 at Daytona in January, 2010. As it concerns (y) and (z) above, access shall be subject to  appropriate security and safety restrictions designated by the applicable racing location and the Racing Team;\n\n\n\n\n\n(iii) making available for the use of the Sponsor a non-racing look-alike (a "Show Car") of the GAINSCO 99 race car (the "Car")  used by the Racing Team. Subject to the Sponsor\'s first right to use the Show Car, it will also be made available to Racing when such  use does not interfere with the Sponsor\'s use of the Show Car;   (iv) allowing the Sponsor the use of the likeness of the Car, including all paint and graphics, for promotion and advertising of or  by the Sponsor, and Racing shall be responsible for all necessary consents and permissions from any other sponsors to be sure the  Sponsor can use the likeness of the Car as specified herein;   (v) prohibiting the endorsement by Racing and any members of Racing, including the drivers, of any entities, products or  services which are in direct competition or otherwise inconsistent with the Sponsor or it products or services, unless such  endorsement activity is approved in writing by Racing and the Sponsor; and   (vi) allowing the Sponsor to use the conference room and other areas of the racing shop and garage for meetings and similar  events, provided that the Sponsor gives prior notice of the need for such use, and such use does not interfere with operations of the  racing shop and garage and is otherwise consistent with reasonable requirements imposed by Racing to assure orderly operations and  provide for adequate safety measures at all times.\n\n3. Sponsorship Fee. The Sponsor shall pay to Racing a sponsorship fee in the amount of $750,000.00 for the term of this Agreement, payable  in an initial installment payable on or before February 1, 2010 in the amount of $350,000.00 and ten installments of $40,000.00 on or before the first  day of each month commencing March 1, 2010 and ending with the installment due on December 1, 2010 (unless this Agreement is sooner  terminated pursuant to Section 14 hereof, in which case Sponsor shall have no obligation to make any payments after the date of termination).\n\n4. Compliance with Applicable Rules and Regulations. Provision of the Benefits pursuant to this Agreement is subject to rules and  requirements of each organization and venue hosting a racing event in which the Racing Team competes during the term hereof, and the Sponsor  agrees to submit to Racing all advertising and other promotional material relating to each such event in sufficient time to enable Racing to assure  compliance with such rules and requirements. If as a result of such rules and requirements Racing is unable to provide a Benefit in the form  requested by the Sponsor, Racing shall be permitted to provide a substitute promotion or advertisement in compliance with such requirements.      2\n\n\n\n\n\n5. Sponsor\'s Maximum Obligation; Indemnification. Racing represents to the Sponsor that the Sponsor\'s aggregate obligation hereunder  will not exceed the amount of the sponsorship fee set forth in Section 3 hereof (or such lesser amount as is payable by the Sponsor in the event  that this Agreement is terminated pursuant to Section 14 hereof), plus, if applicable, collection costs that may be reasonably incurred by Racing in  a legal proceeding to collect all or any part thereof (the "Maximum Obligation"). Racing agrees to indemnify the Sponsor and its officers, directors,  agents and employees and to hold them harmless from any loss, claim, cost, damage or liability in excess of the Maximum Obligation which (i) the  Sponsor shall incur as a result of this Agreement, or (ii) arises from any failure by Racing to perform any of its obligations hereunder.\n\n6. Retention of Rights. The only rights granted to the Sponsor hereunder are the right to receive the Benefits, and Racing hereby retains all  other rights with respect to the Racing Team, including but not limited to logos, symbols, names and other marks and intellectual property of the  Racing Team, and any proceeds derived by the Racing Team. The Sponsor hereby retains and does not grant any rights to Racing to use any of its  logos, symbols, names or other marks or intellectual property, except for use as described in Section 2 hereof. In the event that this Agreement is  terminated or if the sponsorship terminates at the end of the term provided for herein, each of the parties shall retain the rights to use its logos,  symbols, names or other marks or intellectual property including, in the case of the Sponsor, the right to use the names and marks "GAINSCO 99",  "the GAINSCO 99 Car", or similar phrases or derivations thereof.\n\n7. Relationship to Other Sponsors. The Sponsor acknowledges that Racing has arranged and may arrange in the future for other sponsors  for the Racing Team. Racing agrees that, during the term of this Agreement, (i) Sponsor shall have the right to approve or disapprove any  additional sponsor identified by Racing, and (ii) unless another proposed sponsor has agreed to pay a sponsorship fee that exceeds the amount  paid by Sponsor, no other sponsor shall receive any benefit of greater value (including either an equivalent or a more prominent use of another  sponsor\'s name, logo or other identifying information) than the Benefits provided to the Sponsor hereunder.\n\n8. Insurance.   (a) Racing shall obtain and maintain, at Racing\'s expense, comprehensive automobile liability insurance covering all owned, non-owned and  hired vehicles used by Racing in the Business with limits of not less than $5,000,000 per occurrence combined single limit for personal injury and  property damage, including all statutory coverage for all states of operation. Racing shall also provide comprehensive (fire and theft) and collision  insurance on each vehicle used in the Business. Racing shall provide the Sponsor a certificate of insurance evidencing "Gainsco Inc. and all  related entities" as additional insureds, stating that such insurance is primary in coverage to any other insurance which may be available the  Sponsor, and providing at least thirty (30) days\' prior written notice to the Sponsor of cancellation, modification or material change to the policy.\n\n(b) Racing shall obtain and maintain pursuant to the terms of this Agreement, at its sole expense, the following types of insurance coverage,  with minimum limits as set forth below:\n\n(i) Commercial General Liability covering liability arising from premises, operations, independent contractors, personal and advertising injury  and contractual liability—$5,000,000 each occurrence.      3\n\n\n\n\n\n(ii) Racing Owners\' Sponsors (Spectators) Legal Liability including Participant Legal Liability—$5,000,000 each occurrence.\n\n(iii) Business Automobile Liability covering all owned, hired and non-owned vehicles—$5,000,000 each occurrence, including statutory  coverages for all states of operations.\n\n(iv) Workers Compensation—statutory limits for all states of operation.\n\n(v) Employers Liability—$5,000,000 each employee for bodily injury by accident and $500,000 each employee for bodily injury by disease.\n\nAll policies of insurance procured by Racing herein shall be written as primary policies, not contributing with or in excess of coverage that the  Sponsor may carry. If Racing\'s liability policies do not contain the standard separation of insureds provision, or a substantially similar clause, they  shall be endorsed to provide cross-liability coverage.\n\n(c) Racing shall provide the Sponsor with a certificate of insurance evidence compliance with the insurance requirements set forth above.  Certificates shall provide that "Gainsco Inc. and all related entities" shall be named as additional insureds on all liability policies, stating that such  insurance is primary in coverage to any other insurance which may be available to the Sponsor, and providing at least thirty (30) days\' prior  written notice to the Sponsor of termination, cancellation, modification or material change to the policy.\n\n(d) Such certificates shall be in a form acceptable to, and underwritten by insurance company(ies) reasonably satisfactory to the Sponsor. By  requiring insurance herein, the Sponsor does not represent that coverage limits will necessarily be adequate to protect Racing. The purchase of  appropriate insurance coverage by Racing or the furnishing of certificates of insurance shall not release Racing from its obligations and liabilities  under this Agreement.\n\n9. Conduct. Racing and all Racing members, including but not limited to all drivers, agree to use best efforts to conduct themselves in such a  manner so as not to reflect unfavorably upon the Sponsor or its products. The Sponsor shall have the right to terminate this Agreement on written  notice to Racing if any driver, the general manager or any other member of Racing (i) fails to conduct himself/herself in accordance with generally  accepted standards of morality, (ii) engages in any activity which reflects adversely on the image, reputation or goodwill of the Sponsor or (iii)  disparages the products or services of the Sponsor; provided, however, the Sponsor shall not have the right to terminate this Agreement if Racing,  within fifteen (15) days after receipt of written notice by the Sponsor terminates the employment of, or otherwise dismisses from the racing team,  the driver(s), general manager(s) or other member(s) of Racing engaging in the offensive conduct. Upon termination, the Sponsor shall be entitled  to a pro rata refund of monies paid for services not yet performed by Racing based upon the number of races for the applicable racing season. The  Sponsor\'s decision with respect to all matters arising under this Section shall be conclusive.      4\n\n\n\n\n\n10. Remedies. If either party breaches any provision of this Agreement, the other party shall be entitled to seek monetary damages and, if  appropriate, equitable relief to require the performance of the obligations hereunder.\n\n11. Assignment. Neither party shall assign any of its rights or obligations hereunder without the prior written consent of the other party.\n\n12. Entire Agreement; Amendment and Waiver; Confidentiality. This Agreement constitutes the entire agreement between Racing and the  Sponsor with respect to the subject matter hereof and supercedes all prior agreements and understandings. Any amendment of this Agreement  must be by a written instrument signed by both parties, and any waiver of any provision hereof must be in writing, signed by the party agreeing to  such waiver. Each of the parties hereto agrees to hold in confidence the terms hereof and, unless otherwise required by law, neither party shall  release, disclose or publish any of the terms hereof without the prior written consent of the other party.\n\n13. Notices. All notices and communications to be made with respect to this Agreement shall be in writing and shall be effective only when  delivered by (i) hand, (ii) prepaid certified United States mail, return receipt requested, or (iii) overnight delivery service providing proof of delivery,  addressed as follows:\n\nIf to Racing:   Stallings Capital Group Consultants, Ltd., dba Bob Stallings Racing   Attention: Robert W. Stallings, President   4 Windsor Ridge   Frisco, Texas 75034\n\nif to the Sponsor:   GAINSCO, Inc.   Attention: Glenn W. Anderson, President   3333 Lee Parkway, Suite 1200   Dallas, Texas 75219\n\nEither party may change the name or address for notice by providing a written notice of such change in accordance with this Section of the  Agreement.\n\n14. Termination by the Sponsor. Notwithstanding the provisions of Section 1 hereof, the Sponsor shall have the right at any time prior to  December 31, 2010 to terminate this Agreement by giving written notice of such termination to Racing. In the event of such a termination, (i) the  Sponsor shall have no further obligation to make payments toward the sponsorship fee contemplated in Section 3 hereof, (ii) Racing shall have no  further obligation to provide any Benefits hereunder, and (iii) the remaining provisions of this Agreement shall remain in full force and effect.\n\n15. Miscellaneous. (a) This Agreement may be executed in two counterparts, each of which shall be deemed to be an original, but both of  which shall constitute a single agreement.      5\n\n\n\n\n\n(b) The headings and sections of this Agreement are for convenience only and shall not affect the interpretation of any provision hereof.\n\n(c) This Agreement shall be governed and construed in accordance with the internal laws of the State of Texas, without giving effect to  principles of conflict of laws.\n\nThis Agreement is executed as of the date first above written.\n\n   6\n\nSTALLINGS CAPITAL GROUP CONSULTANTS, LTD., DBA BOB  STALLINGS RACING     GAINSCO, INC.\n\nBy:  /s/ Robert W. Stallings     By:  /s/ Glenn W. Anderson    Robert W. Stallings, President       Glenn W. Anderson, President',
  'question': 'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract'},
    {'taskname': 'cuad',
  'context': 'Exhibit 10.41\n\nSPONSORSHIP AGREEMENT\n\nThis Sponsorship Agreement (the "Agreement") is entered into effective January 1, 2010 by and between Stallings Capital Group  Consultants, Ltd., a Texas limited partnership dba Bob Stallings Racing ("Racing"), and GAINSCO, INC., a Texas corporation (the "Sponsor").\n\nRacing organized and operated a racing team engaging in Daytona Prototype Series auto racing (the "Racing Team") in professional races in  2005 through 2008, and the Sponsor was the primary sponsor of the Racing Team pursuant to Sponsorship Agreements dated February 7, 2005,  February 1, 2006, January 1, 2007, January 1, 2008 and January 1, 2009. Racing has invited the Sponsor to continue to act as the primary sponsor of  the Racing Team for 2010, and the Sponsor desires to act in that capacity. In consideration of the sponsorship fee provided for herein, the parties  desire to enter into this Agreement to govern the terms of such sponsorship in 2010.\n\nNow, therefore, Racing and the Sponsor hereby agree as follows:\n\n1. Term. Subject to the provisions of Section 14 hereof, the term of this Agreement and the sponsorship described herein shall commence on  January 1, 2010 and extend through December 31, 2010.\n\n2. Advertising and Other Benefits. Subject to payment by the Sponsor of the sponsorship fee provided for herein, during the term of this  Agreement Racing shall cause the Racing Team to provide for the Sponsor\'s benefit all of the benefits customarily associated with the  sponsorship of a Daytona Prototype Series racing team and consistent with the benefits provided to the Sponsor in 2005 - 2009 (individually, a  "Benefit," and collectively, the "Benefits"), including but not limited to the following:   (i) displaying prominent identification of the Sponsor\'s name and/or logo in signage on the race car and racing suits and, where  appropriate, on other team equipment (subject to approval by the Sponsor);   (ii) making available for the use of the Sponsor (x) the personalities associated with the Racing Team, including without limitation  the name, voice, picture, portrait, likeness, persona and/or signature of each driver for endorsements, commercial advertising and  promotions in any and all media throughout the world during the term of this Agreement, (y) the Racing Team\'s home base facilities in  Texas, and (z) those facilities designated or assigned for the use of the Racing Team at each race and race location at which the Racing  Team actually participates in the race, all for appropriate public relations and other promotional and marketing purposes. Racing agrees  that it will actively participate in the Rolex 24 at Daytona in January, 2010. As it concerns (y) and (z) above, access shall be subject to  appropriate security and safety restrictions designated by the applicable racing location and the Racing Team;\n\n\n\n\n\n(iii) making available for the use of the Sponsor a non-racing look-alike (a "Show Car") of the GAINSCO 99 race car (the "Car")  used by the Racing Team. Subject to the Sponsor\'s first right to use the Show Car, it will also be made available to Racing when such  use does not interfere with the Sponsor\'s use of the Show Car;   (iv) allowing the Sponsor the use of the likeness of the Car, including all paint and graphics, for promotion and advertising of or  by the Sponsor, and Racing shall be responsible for all necessary consents and permissions from any other sponsors to be sure the  Sponsor can use the likeness of the Car as specified herein;   (v) prohibiting the endorsement by Racing and any members of Racing, including the drivers, of any entities, products or  services which are in direct competition or otherwise inconsistent with the Sponsor or it products or services, unless such  endorsement activity is approved in writing by Racing and the Sponsor; and   (vi) allowing the Sponsor to use the conference room and other areas of the racing shop and garage for meetings and similar  events, provided that the Sponsor gives prior notice of the need for such use, and such use does not interfere with operations of the  racing shop and garage and is otherwise consistent with reasonable requirements imposed by Racing to assure orderly operations and  provide for adequate safety measures at all times.\n\n3. Sponsorship Fee. The Sponsor shall pay to Racing a sponsorship fee in the amount of $750,000.00 for the term of this Agreement, payable  in an initial installment payable on or before February 1, 2010 in the amount of $350,000.00 and ten installments of $40,000.00 on or before the first  day of each month commencing March 1, 2010 and ending with the installment due on December 1, 2010 (unless this Agreement is sooner  terminated pursuant to Section 14 hereof, in which case Sponsor shall have no obligation to make any payments after the date of termination).\n\n4. Compliance with Applicable Rules and Regulations. Provision of the Benefits pursuant to this Agreement is subject to rules and  requirements of each organization and venue hosting a racing event in which the Racing Team competes during the term hereof, and the Sponsor  agrees to submit to Racing all advertising and other promotional material relating to each such event in sufficient time to enable Racing to assure  compliance with such rules and requirements. If as a result of such rules and requirements Racing is unable to provide a Benefit in the form  requested by the Sponsor, Racing shall be permitted to provide a substitute promotion or advertisement in compliance with such requirements.      2\n\n\n\n\n\n5. Sponsor\'s Maximum Obligation; Indemnification. Racing represents to the Sponsor that the Sponsor\'s aggregate obligation hereunder  will not exceed the amount of the sponsorship fee set forth in Section 3 hereof (or such lesser amount as is payable by the Sponsor in the event  that this Agreement is terminated pursuant to Section 14 hereof), plus, if applicable, collection costs that may be reasonably incurred by Racing in  a legal proceeding to collect all or any part thereof (the "Maximum Obligation"). Racing agrees to indemnify the Sponsor and its officers, directors,  agents and employees and to hold them harmless from any loss, claim, cost, damage or liability in excess of the Maximum Obligation which (i) the  Sponsor shall incur as a result of this Agreement, or (ii) arises from any failure by Racing to perform any of its obligations hereunder.\n\n6. Retention of Rights. The only rights granted to the Sponsor hereunder are the right to receive the Benefits, and Racing hereby retains all  other rights with respect to the Racing Team, including but not limited to logos, symbols, names and other marks and intellectual property of the  Racing Team, and any proceeds derived by the Racing Team. The Sponsor hereby retains and does not grant any rights to Racing to use any of its  logos, symbols, names or other marks or intellectual property, except for use as described in Section 2 hereof. In the event that this Agreement is  terminated or if the sponsorship terminates at the end of the term provided for herein, each of the parties shall retain the rights to use its logos,  symbols, names or other marks or intellectual property including, in the case of the Sponsor, the right to use the names and marks "GAINSCO 99",  "the GAINSCO 99 Car", or similar phrases or derivations thereof.\n\n7. Relationship to Other Sponsors. The Sponsor acknowledges that Racing has arranged and may arrange in the future for other sponsors  for the Racing Team. Racing agrees that, during the term of this Agreement, (i) Sponsor shall have the right to approve or disapprove any  additional sponsor identified by Racing, and (ii) unless another proposed sponsor has agreed to pay a sponsorship fee that exceeds the amount  paid by Sponsor, no other sponsor shall receive any benefit of greater value (including either an equivalent or a more prominent use of another  sponsor\'s name, logo or other identifying information) than the Benefits provided to the Sponsor hereunder.\n\n8. Insurance.   (a) Racing shall obtain and maintain, at Racing\'s expense, comprehensive automobile liability insurance covering all owned, non-owned and  hired vehicles used by Racing in the Business with limits of not less than $5,000,000 per occurrence combined single limit for personal injury and  property damage, including all statutory coverage for all states of operation. Racing shall also provide comprehensive (fire and theft) and collision  insurance on each vehicle used in the Business. Racing shall provide the Sponsor a certificate of insurance evidencing "Gainsco Inc. and all  related entities" as additional insureds, stating that such insurance is primary in coverage to any other insurance which may be available the  Sponsor, and providing at least thirty (30) days\' prior written notice to the Sponsor of cancellation, modification or material change to the policy.\n\n(b) Racing shall obtain and maintain pursuant to the terms of this Agreement, at its sole expense, the following types of insurance coverage,  with minimum limits as set forth below:\n\n(i) Commercial General Liability covering liability arising from premises, operations, independent contractors, personal and advertising injury  and contractual liability—$5,000,000 each occurrence.      3\n\n\n\n\n\n(ii) Racing Owners\' Sponsors (Spectators) Legal Liability including Participant Legal Liability—$5,000,000 each occurrence.\n\n(iii) Business Automobile Liability covering all owned, hired and non-owned vehicles—$5,000,000 each occurrence, including statutory  coverages for all states of operations.\n\n(iv) Workers Compensation—statutory limits for all states of operation.\n\n(v) Employers Liability—$5,000,000 each employee for bodily injury by accident and $500,000 each employee for bodily injury by disease.\n\nAll policies of insurance procured by Racing herein shall be written as primary policies, not contributing with or in excess of coverage that the  Sponsor may carry. If Racing\'s liability policies do not contain the standard separation of insureds provision, or a substantially similar clause, they  shall be endorsed to provide cross-liability coverage.\n\n(c) Racing shall provide the Sponsor with a certificate of insurance evidence compliance with the insurance requirements set forth above.  Certificates shall provide that "Gainsco Inc. and all related entities" shall be named as additional insureds on all liability policies, stating that such  insurance is primary in coverage to any other insurance which may be available to the Sponsor, and providing at least thirty (30) days\' prior  written notice to the Sponsor of termination, cancellation, modification or material change to the policy.\n\n(d) Such certificates shall be in a form acceptable to, and underwritten by insurance company(ies) reasonably satisfactory to the Sponsor. By  requiring insurance herein, the Sponsor does not represent that coverage limits will necessarily be adequate to protect Racing. The purchase of  appropriate insurance coverage by Racing or the furnishing of certificates of insurance shall not release Racing from its obligations and liabilities  under this Agreement.\n\n9. Conduct. Racing and all Racing members, including but not limited to all drivers, agree to use best efforts to conduct themselves in such a  manner so as not to reflect unfavorably upon the Sponsor or its products. The Sponsor shall have the right to terminate this Agreement on written  notice to Racing if any driver, the general manager or any other member of Racing (i) fails to conduct himself/herself in accordance with generally  accepted standards of morality, (ii) engages in any activity which reflects adversely on the image, reputation or goodwill of the Sponsor or (iii)  disparages the products or services of the Sponsor; provided, however, the Sponsor shall not have the right to terminate this Agreement if Racing,  within fifteen (15) days after receipt of written notice by the Sponsor terminates the employment of, or otherwise dismisses from the racing team,  the driver(s), general manager(s) or other member(s) of Racing engaging in the offensive conduct. Upon termination, the Sponsor shall be entitled  to a pro rata refund of monies paid for services not yet performed by Racing based upon the number of races for the applicable racing season. The  Sponsor\'s decision with respect to all matters arising under this Section shall be conclusive.      4\n\n\n\n\n\n10. Remedies. If either party breaches any provision of this Agreement, the other party shall be entitled to seek monetary damages and, if  appropriate, equitable relief to require the performance of the obligations hereunder.\n\n11. Assignment. Neither party shall assign any of its rights or obligations hereunder without the prior written consent of the other party.\n\n12. Entire Agreement; Amendment and Waiver; Confidentiality. This Agreement constitutes the entire agreement between Racing and the  Sponsor with respect to the subject matter hereof and supercedes all prior agreements and understandings. Any amendment of this Agreement  must be by a written instrument signed by both parties, and any waiver of any provision hereof must be in writing, signed by the party agreeing to  such waiver. Each of the parties hereto agrees to hold in confidence the terms hereof and, unless otherwise required by law, neither party shall  release, disclose or publish any of the terms hereof without the prior written consent of the other party.\n\n13. Notices. All notices and communications to be made with respect to this Agreement shall be in writing and shall be effective only when  delivered by (i) hand, (ii) prepaid certified United States mail, return receipt requested, or (iii) overnight delivery service providing proof of delivery,  addressed as follows:\n\nIf to Racing:   Stallings Capital Group Consultants, Ltd., dba Bob Stallings Racing   Attention: Robert W. Stallings, President   4 Windsor Ridge   Frisco, Texas 75034\n\nif to the Sponsor:   GAINSCO, Inc.   Attention: Glenn W. Anderson, President   3333 Lee Parkway, Suite 1200   Dallas, Texas 75219\n\nEither party may change the name or address for notice by providing a written notice of such change in accordance with this Section of the  Agreement.\n\n14. Termination by the Sponsor. Notwithstanding the provisions of Section 1 hereof, the Sponsor shall have the right at any time prior to  December 31, 2010 to terminate this Agreement by giving written notice of such termination to Racing. In the event of such a termination, (i) the  Sponsor shall have no further obligation to make payments toward the sponsorship fee contemplated in Section 3 hereof, (ii) Racing shall have no  further obligation to provide any Benefits hereunder, and (iii) the remaining provisions of this Agreement shall remain in full force and effect.\n\n15. Miscellaneous. (a) This Agreement may be executed in two counterparts, each of which shall be deemed to be an original, but both of  which shall constitute a single agreement.      5\n\n\n\n\n\n(b) The headings and sections of this Agreement are for convenience only and shall not affect the interpretation of any provision hereof.\n\n(c) This Agreement shall be governed and construed in accordance with the internal laws of the State of Texas, without giving effect to  principles of conflict of laws.\n\nThis Agreement is executed as of the date first above written.\n\n   6\n\nSTALLINGS CAPITAL GROUP CONSULTANTS, LTD., DBA BOB  STALLINGS RACING     GAINSCO, INC.\n\nBy:  /s/ Robert W. Stallings     By:  /s/ Glenn W. Anderson    Robert W. Stallings, President       Glenn W. Anderson, President',
  'question': 'Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract'},
    {'taskname': 'cuad',
  'context': 'Exhibit 10.41\n\nSPONSORSHIP AGREEMENT\n\nThis Sponsorship Agreement (the "Agreement") is entered into effective January 1, 2010 by and between Stallings Capital Group  Consultants, Ltd., a Texas limited partnership dba Bob Stallings Racing ("Racing"), and GAINSCO, INC., a Texas corporation (the "Sponsor").\n\nRacing organized and operated a racing team engaging in Daytona Prototype Series auto racing (the "Racing Team") in professional races in  2005 through 2008, and the Sponsor was the primary sponsor of the Racing Team pursuant to Sponsorship Agreements dated February 7, 2005,  February 1, 2006, January 1, 2007, January 1, 2008 and January 1, 2009. Racing has invited the Sponsor to continue to act as the primary sponsor of  the Racing Team for 2010, and the Sponsor desires to act in that capacity. In consideration of the sponsorship fee provided for herein, the parties  desire to enter into this Agreement to govern the terms of such sponsorship in 2010.\n\nNow, therefore, Racing and the Sponsor hereby agree as follows:\n\n1. Term. Subject to the provisions of Section 14 hereof, the term of this Agreement and the sponsorship described herein shall commence on  January 1, 2010 and extend through December 31, 2010.\n\n2. Advertising and Other Benefits. Subject to payment by the Sponsor of the sponsorship fee provided for herein, during the term of this  Agreement Racing shall cause the Racing Team to provide for the Sponsor\'s benefit all of the benefits customarily associated with the  sponsorship of a Daytona Prototype Series racing team and consistent with the benefits provided to the Sponsor in 2005 - 2009 (individually, a  "Benefit," and collectively, the "Benefits"), including but not limited to the following:   (i) displaying prominent identification of the Sponsor\'s name and/or logo in signage on the race car and racing suits and, where  appropriate, on other team equipment (subject to approval by the Sponsor);   (ii) making available for the use of the Sponsor (x) the personalities associated with the Racing Team, including without limitation  the name, voice, picture, portrait, likeness, persona and/or signature of each driver for endorsements, commercial advertising and  promotions in any and all media throughout the world during the term of this Agreement, (y) the Racing Team\'s home base facilities in  Texas, and (z) those facilities designated or assigned for the use of the Racing Team at each race and race location at which the Racing  Team actually participates in the race, all for appropriate public relations and other promotional and marketing purposes. Racing agrees  that it will actively participate in the Rolex 24 at Daytona in January, 2010. As it concerns (y) and (z) above, access shall be subject to  appropriate security and safety restrictions designated by the applicable racing location and the Racing Team;\n\n\n\n\n\n(iii) making available for the use of the Sponsor a non-racing look-alike (a "Show Car") of the GAINSCO 99 race car (the "Car")  used by the Racing Team. Subject to the Sponsor\'s first right to use the Show Car, it will also be made available to Racing when such  use does not interfere with the Sponsor\'s use of the Show Car;   (iv) allowing the Sponsor the use of the likeness of the Car, including all paint and graphics, for promotion and advertising of or  by the Sponsor, and Racing shall be responsible for all necessary consents and permissions from any other sponsors to be sure the  Sponsor can use the likeness of the Car as specified herein;   (v) prohibiting the endorsement by Racing and any members of Racing, including the drivers, of any entities, products or  services which are in direct competition or otherwise inconsistent with the Sponsor or it products or services, unless such  endorsement activity is approved in writing by Racing and the Sponsor; and   (vi) allowing the Sponsor to use the conference room and other areas of the racing shop and garage for meetings and similar  events, provided that the Sponsor gives prior notice of the need for such use, and such use does not interfere with operations of the  racing shop and garage and is otherwise consistent with reasonable requirements imposed by Racing to assure orderly operations and  provide for adequate safety measures at all times.\n\n3. Sponsorship Fee. The Sponsor shall pay to Racing a sponsorship fee in the amount of $750,000.00 for the term of this Agreement, payable  in an initial installment payable on or before February 1, 2010 in the amount of $350,000.00 and ten installments of $40,000.00 on or before the first  day of each month commencing March 1, 2010 and ending with the installment due on December 1, 2010 (unless this Agreement is sooner  terminated pursuant to Section 14 hereof, in which case Sponsor shall have no obligation to make any payments after the date of termination).\n\n4. Compliance with Applicable Rules and Regulations. Provision of the Benefits pursuant to this Agreement is subject to rules and  requirements of each organization and venue hosting a racing event in which the Racing Team competes during the term hereof, and the Sponsor  agrees to submit to Racing all advertising and other promotional material relating to each such event in sufficient time to enable Racing to assure  compliance with such rules and requirements. If as a result of such rules and requirements Racing is unable to provide a Benefit in the form  requested by the Sponsor, Racing shall be permitted to provide a substitute promotion or advertisement in compliance with such requirements.      2\n\n\n\n\n\n5. Sponsor\'s Maximum Obligation; Indemnification. Racing represents to the Sponsor that the Sponsor\'s aggregate obligation hereunder  will not exceed the amount of the sponsorship fee set forth in Section 3 hereof (or such lesser amount as is payable by the Sponsor in the event  that this Agreement is terminated pursuant to Section 14 hereof), plus, if applicable, collection costs that may be reasonably incurred by Racing in  a legal proceeding to collect all or any part thereof (the "Maximum Obligation"). Racing agrees to indemnify the Sponsor and its officers, directors,  agents and employees and to hold them harmless from any loss, claim, cost, damage or liability in excess of the Maximum Obligation which (i) the  Sponsor shall incur as a result of this Agreement, or (ii) arises from any failure by Racing to perform any of its obligations hereunder.\n\n6. Retention of Rights. The only rights granted to the Sponsor hereunder are the right to receive the Benefits, and Racing hereby retains all  other rights with respect to the Racing Team, including but not limited to logos, symbols, names and other marks and intellectual property of the  Racing Team, and any proceeds derived by the Racing Team. The Sponsor hereby retains and does not grant any rights to Racing to use any of its  logos, symbols, names or other marks or intellectual property, except for use as described in Section 2 hereof. In the event that this Agreement is  terminated or if the sponsorship terminates at the end of the term provided for herein, each of the parties shall retain the rights to use its logos,  symbols, names or other marks or intellectual property including, in the case of the Sponsor, the right to use the names and marks "GAINSCO 99",  "the GAINSCO 99 Car", or similar phrases or derivations thereof.\n\n7. Relationship to Other Sponsors. The Sponsor acknowledges that Racing has arranged and may arrange in the future for other sponsors  for the Racing Team. Racing agrees that, during the term of this Agreement, (i) Sponsor shall have the right to approve or disapprove any  additional sponsor identified by Racing, and (ii) unless another proposed sponsor has agreed to pay a sponsorship fee that exceeds the amount  paid by Sponsor, no other sponsor shall receive any benefit of greater value (including either an equivalent or a more prominent use of another  sponsor\'s name, logo or other identifying information) than the Benefits provided to the Sponsor hereunder.\n\n8. Insurance.   (a) Racing shall obtain and maintain, at Racing\'s expense, comprehensive automobile liability insurance covering all owned, non-owned and  hired vehicles used by Racing in the Business with limits of not less than $5,000,000 per occurrence combined single limit for personal injury and  property damage, including all statutory coverage for all states of operation. Racing shall also provide comprehensive (fire and theft) and collision  insurance on each vehicle used in the Business. Racing shall provide the Sponsor a certificate of insurance evidencing "Gainsco Inc. and all  related entities" as additional insureds, stating that such insurance is primary in coverage to any other insurance which may be available the  Sponsor, and providing at least thirty (30) days\' prior written notice to the Sponsor of cancellation, modification or material change to the policy.\n\n(b) Racing shall obtain and maintain pursuant to the terms of this Agreement, at its sole expense, the following types of insurance coverage,  with minimum limits as set forth below:\n\n(i) Commercial General Liability covering liability arising from premises, operations, independent contractors, personal and advertising injury  and contractual liability—$5,000,000 each occurrence.      3\n\n\n\n\n\n(ii) Racing Owners\' Sponsors (Spectators) Legal Liability including Participant Legal Liability—$5,000,000 each occurrence.\n\n(iii) Business Automobile Liability covering all owned, hired and non-owned vehicles—$5,000,000 each occurrence, including statutory  coverages for all states of operations.\n\n(iv) Workers Compensation—statutory limits for all states of operation.\n\n(v) Employers Liability—$5,000,000 each employee for bodily injury by accident and $500,000 each employee for bodily injury by disease.\n\nAll policies of insurance procured by Racing herein shall be written as primary policies, not contributing with or in excess of coverage that the  Sponsor may carry. If Racing\'s liability policies do not contain the standard separation of insureds provision, or a substantially similar clause, they  shall be endorsed to provide cross-liability coverage.\n\n(c) Racing shall provide the Sponsor with a certificate of insurance evidence compliance with the insurance requirements set forth above.  Certificates shall provide that "Gainsco Inc. and all related entities" shall be named as additional insureds on all liability policies, stating that such  insurance is primary in coverage to any other insurance which may be available to the Sponsor, and providing at least thirty (30) days\' prior  written notice to the Sponsor of termination, cancellation, modification or material change to the policy.\n\n(d) Such certificates shall be in a form acceptable to, and underwritten by insurance company(ies) reasonably satisfactory to the Sponsor. By  requiring insurance herein, the Sponsor does not represent that coverage limits will necessarily be adequate to protect Racing. The purchase of  appropriate insurance coverage by Racing or the furnishing of certificates of insurance shall not release Racing from its obligations and liabilities  under this Agreement.\n\n9. Conduct. Racing and all Racing members, including but not limited to all drivers, agree to use best efforts to conduct themselves in such a  manner so as not to reflect unfavorably upon the Sponsor or its products. The Sponsor shall have the right to terminate this Agreement on written  notice to Racing if any driver, the general manager or any other member of Racing (i) fails to conduct himself/herself in accordance with generally  accepted standards of morality, (ii) engages in any activity which reflects adversely on the image, reputation or goodwill of the Sponsor or (iii)  disparages the products or services of the Sponsor; provided, however, the Sponsor shall not have the right to terminate this Agreement if Racing,  within fifteen (15) days after receipt of written notice by the Sponsor terminates the employment of, or otherwise dismisses from the racing team,  the driver(s), general manager(s) or other member(s) of Racing engaging in the offensive conduct. Upon termination, the Sponsor shall be entitled  to a pro rata refund of monies paid for services not yet performed by Racing based upon the number of races for the applicable racing season. The  Sponsor\'s decision with respect to all matters arising under this Section shall be conclusive.      4\n\n\n\n\n\n10. Remedies. If either party breaches any provision of this Agreement, the other party shall be entitled to seek monetary damages and, if  appropriate, equitable relief to require the performance of the obligations hereunder.\n\n11. Assignment. Neither party shall assign any of its rights or obligations hereunder without the prior written consent of the other party.\n\n12. Entire Agreement; Amendment and Waiver; Confidentiality. This Agreement constitutes the entire agreement between Racing and the  Sponsor with respect to the subject matter hereof and supercedes all prior agreements and understandings. Any amendment of this Agreement  must be by a written instrument signed by both parties, and any waiver of any provision hereof must be in writing, signed by the party agreeing to  such waiver. Each of the parties hereto agrees to hold in confidence the terms hereof and, unless otherwise required by law, neither party shall  release, disclose or publish any of the terms hereof without the prior written consent of the other party.\n\n13. Notices. All notices and communications to be made with respect to this Agreement shall be in writing and shall be effective only when  delivered by (i) hand, (ii) prepaid certified United States mail, return receipt requested, or (iii) overnight delivery service providing proof of delivery,  addressed as follows:\n\nIf to Racing:   Stallings Capital Group Consultants, Ltd., dba Bob Stallings Racing   Attention: Robert W. Stallings, President   4 Windsor Ridge   Frisco, Texas 75034\n\nif to the Sponsor:   GAINSCO, Inc.   Attention: Glenn W. Anderson, President   3333 Lee Parkway, Suite 1200   Dallas, Texas 75219\n\nEither party may change the name or address for notice by providing a written notice of such change in accordance with this Section of the  Agreement.\n\n14. Termination by the Sponsor. Notwithstanding the provisions of Section 1 hereof, the Sponsor shall have the right at any time prior to  December 31, 2010 to terminate this Agreement by giving written notice of such termination to Racing. In the event of such a termination, (i) the  Sponsor shall have no further obligation to make payments toward the sponsorship fee contemplated in Section 3 hereof, (ii) Racing shall have no  further obligation to provide any Benefits hereunder, and (iii) the remaining provisions of this Agreement shall remain in full force and effect.\n\n15. Miscellaneous. (a) This Agreement may be executed in two counterparts, each of which shall be deemed to be an original, but both of  which shall constitute a single agreement.      5\n\n\n\n\n\n(b) The headings and sections of this Agreement are for convenience only and shall not affect the interpretation of any provision hereof.\n\n(c) This Agreement shall be governed and construed in accordance with the internal laws of the State of Texas, without giving effect to  principles of conflict of laws.\n\nThis Agreement is executed as of the date first above written.\n\n   6\n\nSTALLINGS CAPITAL GROUP CONSULTANTS, LTD., DBA BOB  STALLINGS RACING     GAINSCO, INC.\n\nBy:  /s/ Robert W. Stallings     By:  /s/ Glenn W. Anderson    Robert W. Stallings, President       Glenn W. Anderson, President',
  'question': 'Highlight the parts (if any) of this contract related to "Effective Date" that should be reviewed by a lawyer. Details: The date when the contract is effective\xa0'},
    {'taskname': 'cuad',
  'context': 'Exhibit 10.41\n\nSPONSORSHIP AGREEMENT\n\nThis Sponsorship Agreement (the "Agreement") is entered into effective January 1, 2010 by and between Stallings Capital Group  Consultants, Ltd., a Texas limited partnership dba Bob Stallings Racing ("Racing"), and GAINSCO, INC., a Texas corporation (the "Sponsor").\n\nRacing organized and operated a racing team engaging in Daytona Prototype Series auto racing (the "Racing Team") in professional races in  2005 through 2008, and the Sponsor was the primary sponsor of the Racing Team pursuant to Sponsorship Agreements dated February 7, 2005,  February 1, 2006, January 1, 2007, January 1, 2008 and January 1, 2009. Racing has invited the Sponsor to continue to act as the primary sponsor of  the Racing Team for 2010, and the Sponsor desires to act in that capacity. In consideration of the sponsorship fee provided for herein, the parties  desire to enter into this Agreement to govern the terms of such sponsorship in 2010.\n\nNow, therefore, Racing and the Sponsor hereby agree as follows:\n\n1. Term. Subject to the provisions of Section 14 hereof, the term of this Agreement and the sponsorship described herein shall commence on  January 1, 2010 and extend through December 31, 2010.\n\n2. Advertising and Other Benefits. Subject to payment by the Sponsor of the sponsorship fee provided for herein, during the term of this  Agreement Racing shall cause the Racing Team to provide for the Sponsor\'s benefit all of the benefits customarily associated with the  sponsorship of a Daytona Prototype Series racing team and consistent with the benefits provided to the Sponsor in 2005 - 2009 (individually, a  "Benefit," and collectively, the "Benefits"), including but not limited to the following:   (i) displaying prominent identification of the Sponsor\'s name and/or logo in signage on the race car and racing suits and, where  appropriate, on other team equipment (subject to approval by the Sponsor);   (ii) making available for the use of the Sponsor (x) the personalities associated with the Racing Team, including without limitation  the name, voice, picture, portrait, likeness, persona and/or signature of each driver for endorsements, commercial advertising and  promotions in any and all media throughout the world during the term of this Agreement, (y) the Racing Team\'s home base facilities in  Texas, and (z) those facilities designated or assigned for the use of the Racing Team at each race and race location at which the Racing  Team actually participates in the race, all for appropriate public relations and other promotional and marketing purposes. Racing agrees  that it will actively participate in the Rolex 24 at Daytona in January, 2010. As it concerns (y) and (z) above, access shall be subject to  appropriate security and safety restrictions designated by the applicable racing location and the Racing Team;\n\n\n\n\n\n(iii) making available for the use of the Sponsor a non-racing look-alike (a "Show Car") of the GAINSCO 99 race car (the "Car")  used by the Racing Team. Subject to the Sponsor\'s first right to use the Show Car, it will also be made available to Racing when such  use does not interfere with the Sponsor\'s use of the Show Car;   (iv) allowing the Sponsor the use of the likeness of the Car, including all paint and graphics, for promotion and advertising of or  by the Sponsor, and Racing shall be responsible for all necessary consents and permissions from any other sponsors to be sure the  Sponsor can use the likeness of the Car as specified herein;   (v) prohibiting the endorsement by Racing and any members of Racing, including the drivers, of any entities, products or  services which are in direct competition or otherwise inconsistent with the Sponsor or it products or services, unless such  endorsement activity is approved in writing by Racing and the Sponsor; and   (vi) allowing the Sponsor to use the conference room and other areas of the racing shop and garage for meetings and similar  events, provided that the Sponsor gives prior notice of the need for such use, and such use does not interfere with operations of the  racing shop and garage and is otherwise consistent with reasonable requirements imposed by Racing to assure orderly operations and  provide for adequate safety measures at all times.\n\n3. Sponsorship Fee. The Sponsor shall pay to Racing a sponsorship fee in the amount of $750,000.00 for the term of this Agreement, payable  in an initial installment payable on or before February 1, 2010 in the amount of $350,000.00 and ten installments of $40,000.00 on or before the first  day of each month commencing March 1, 2010 and ending with the installment due on December 1, 2010 (unless this Agreement is sooner  terminated pursuant to Section 14 hereof, in which case Sponsor shall have no obligation to make any payments after the date of termination).\n\n4. Compliance with Applicable Rules and Regulations. Provision of the Benefits pursuant to this Agreement is subject to rules and  requirements of each organization and venue hosting a racing event in which the Racing Team competes during the term hereof, and the Sponsor  agrees to submit to Racing all advertising and other promotional material relating to each such event in sufficient time to enable Racing to assure  compliance with such rules and requirements. If as a result of such rules and requirements Racing is unable to provide a Benefit in the form  requested by the Sponsor, Racing shall be permitted to provide a substitute promotion or advertisement in compliance with such requirements.      2\n\n\n\n\n\n5. Sponsor\'s Maximum Obligation; Indemnification. Racing represents to the Sponsor that the Sponsor\'s aggregate obligation hereunder  will not exceed the amount of the sponsorship fee set forth in Section 3 hereof (or such lesser amount as is payable by the Sponsor in the event  that this Agreement is terminated pursuant to Section 14 hereof), plus, if applicable, collection costs that may be reasonably incurred by Racing in  a legal proceeding to collect all or any part thereof (the "Maximum Obligation"). Racing agrees to indemnify the Sponsor and its officers, directors,  agents and employees and to hold them harmless from any loss, claim, cost, damage or liability in excess of the Maximum Obligation which (i) the  Sponsor shall incur as a result of this Agreement, or (ii) arises from any failure by Racing to perform any of its obligations hereunder.\n\n6. Retention of Rights. The only rights granted to the Sponsor hereunder are the right to receive the Benefits, and Racing hereby retains all  other rights with respect to the Racing Team, including but not limited to logos, symbols, names and other marks and intellectual property of the  Racing Team, and any proceeds derived by the Racing Team. The Sponsor hereby retains and does not grant any rights to Racing to use any of its  logos, symbols, names or other marks or intellectual property, except for use as described in Section 2 hereof. In the event that this Agreement is  terminated or if the sponsorship terminates at the end of the term provided for herein, each of the parties shall retain the rights to use its logos,  symbols, names or other marks or intellectual property including, in the case of the Sponsor, the right to use the names and marks "GAINSCO 99",  "the GAINSCO 99 Car", or similar phrases or derivations thereof.\n\n7. Relationship to Other Sponsors. The Sponsor acknowledges that Racing has arranged and may arrange in the future for other sponsors  for the Racing Team. Racing agrees that, during the term of this Agreement, (i) Sponsor shall have the right to approve or disapprove any  additional sponsor identified by Racing, and (ii) unless another proposed sponsor has agreed to pay a sponsorship fee that exceeds the amount  paid by Sponsor, no other sponsor shall receive any benefit of greater value (including either an equivalent or a more prominent use of another  sponsor\'s name, logo or other identifying information) than the Benefits provided to the Sponsor hereunder.\n\n8. Insurance.   (a) Racing shall obtain and maintain, at Racing\'s expense, comprehensive automobile liability insurance covering all owned, non-owned and  hired vehicles used by Racing in the Business with limits of not less than $5,000,000 per occurrence combined single limit for personal injury and  property damage, including all statutory coverage for all states of operation. Racing shall also provide comprehensive (fire and theft) and collision  insurance on each vehicle used in the Business. Racing shall provide the Sponsor a certificate of insurance evidencing "Gainsco Inc. and all  related entities" as additional insureds, stating that such insurance is primary in coverage to any other insurance which may be available the  Sponsor, and providing at least thirty (30) days\' prior written notice to the Sponsor of cancellation, modification or material change to the policy.\n\n(b) Racing shall obtain and maintain pursuant to the terms of this Agreement, at its sole expense, the following types of insurance coverage,  with minimum limits as set forth below:\n\n(i) Commercial General Liability covering liability arising from premises, operations, independent contractors, personal and advertising injury  and contractual liability—$5,000,000 each occurrence.      3\n\n\n\n\n\n(ii) Racing Owners\' Sponsors (Spectators) Legal Liability including Participant Legal Liability—$5,000,000 each occurrence.\n\n(iii) Business Automobile Liability covering all owned, hired and non-owned vehicles—$5,000,000 each occurrence, including statutory  coverages for all states of operations.\n\n(iv) Workers Compensation—statutory limits for all states of operation.\n\n(v) Employers Liability—$5,000,000 each employee for bodily injury by accident and $500,000 each employee for bodily injury by disease.\n\nAll policies of insurance procured by Racing herein shall be written as primary policies, not contributing with or in excess of coverage that the  Sponsor may carry. If Racing\'s liability policies do not contain the standard separation of insureds provision, or a substantially similar clause, they  shall be endorsed to provide cross-liability coverage.\n\n(c) Racing shall provide the Sponsor with a certificate of insurance evidence compliance with the insurance requirements set forth above.  Certificates shall provide that "Gainsco Inc. and all related entities" shall be named as additional insureds on all liability policies, stating that such  insurance is primary in coverage to any other insurance which may be available to the Sponsor, and providing at least thirty (30) days\' prior  written notice to the Sponsor of termination, cancellation, modification or material change to the policy.\n\n(d) Such certificates shall be in a form acceptable to, and underwritten by insurance company(ies) reasonably satisfactory to the Sponsor. By  requiring insurance herein, the Sponsor does not represent that coverage limits will necessarily be adequate to protect Racing. The purchase of  appropriate insurance coverage by Racing or the furnishing of certificates of insurance shall not release Racing from its obligations and liabilities  under this Agreement.\n\n9. Conduct. Racing and all Racing members, including but not limited to all drivers, agree to use best efforts to conduct themselves in such a  manner so as not to reflect unfavorably upon the Sponsor or its products. The Sponsor shall have the right to terminate this Agreement on written  notice to Racing if any driver, the general manager or any other member of Racing (i) fails to conduct himself/herself in accordance with generally  accepted standards of morality, (ii) engages in any activity which reflects adversely on the image, reputation or goodwill of the Sponsor or (iii)  disparages the products or services of the Sponsor; provided, however, the Sponsor shall not have the right to terminate this Agreement if Racing,  within fifteen (15) days after receipt of written notice by the Sponsor terminates the employment of, or otherwise dismisses from the racing team,  the driver(s), general manager(s) or other member(s) of Racing engaging in the offensive conduct. Upon termination, the Sponsor shall be entitled  to a pro rata refund of monies paid for services not yet performed by Racing based upon the number of races for the applicable racing season. The  Sponsor\'s decision with respect to all matters arising under this Section shall be conclusive.      4\n\n\n\n\n\n10. Remedies. If either party breaches any provision of this Agreement, the other party shall be entitled to seek monetary damages and, if  appropriate, equitable relief to require the performance of the obligations hereunder.\n\n11. Assignment. Neither party shall assign any of its rights or obligations hereunder without the prior written consent of the other party.\n\n12. Entire Agreement; Amendment and Waiver; Confidentiality. This Agreement constitutes the entire agreement between Racing and the  Sponsor with respect to the subject matter hereof and supercedes all prior agreements and understandings. Any amendment of this Agreement  must be by a written instrument signed by both parties, and any waiver of any provision hereof must be in writing, signed by the party agreeing to  such waiver. Each of the parties hereto agrees to hold in confidence the terms hereof and, unless otherwise required by law, neither party shall  release, disclose or publish any of the terms hereof without the prior written consent of the other party.\n\n13. Notices. All notices and communications to be made with respect to this Agreement shall be in writing and shall be effective only when  delivered by (i) hand, (ii) prepaid certified United States mail, return receipt requested, or (iii) overnight delivery service providing proof of delivery,  addressed as follows:\n\nIf to Racing:   Stallings Capital Group Consultants, Ltd., dba Bob Stallings Racing   Attention: Robert W. Stallings, President   4 Windsor Ridge   Frisco, Texas 75034\n\nif to the Sponsor:   GAINSCO, Inc.   Attention: Glenn W. Anderson, President   3333 Lee Parkway, Suite 1200   Dallas, Texas 75219\n\nEither party may change the name or address for notice by providing a written notice of such change in accordance with this Section of the  Agreement.\n\n14. Termination by the Sponsor. Notwithstanding the provisions of Section 1 hereof, the Sponsor shall have the right at any time prior to  December 31, 2010 to terminate this Agreement by giving written notice of such termination to Racing. In the event of such a termination, (i) the  Sponsor shall have no further obligation to make payments toward the sponsorship fee contemplated in Section 3 hereof, (ii) Racing shall have no  further obligation to provide any Benefits hereunder, and (iii) the remaining provisions of this Agreement shall remain in full force and effect.\n\n15. Miscellaneous. (a) This Agreement may be executed in two counterparts, each of which shall be deemed to be an original, but both of  which shall constitute a single agreement.      5\n\n\n\n\n\n(b) The headings and sections of this Agreement are for convenience only and shall not affect the interpretation of any provision hereof.\n\n(c) This Agreement shall be governed and construed in accordance with the internal laws of the State of Texas, without giving effect to  principles of conflict of laws.\n\nThis Agreement is executed as of the date first above written.\n\n   6\n\nSTALLINGS CAPITAL GROUP CONSULTANTS, LTD., DBA BOB  STALLINGS RACING     GAINSCO, INC.\n\nBy:  /s/ Robert W. Stallings     By:  /s/ Glenn W. Anderson    Robert W. Stallings, President       Glenn W. Anderson, President',
  'question': 'Highlight the parts (if any) of this contract related to "Expiration Date" that should be reviewed by a lawyer. Details: On what date will the contract\'s initial term expire?'},
    {'taskname': 'cuad',
  'context': 'Exhibit 10.41\n\nSPONSORSHIP AGREEMENT\n\nThis Sponsorship Agreement (the "Agreement") is entered into effective January 1, 2010 by and between Stallings Capital Group  Consultants, Ltd., a Texas limited partnership dba Bob Stallings Racing ("Racing"), and GAINSCO, INC., a Texas corporation (the "Sponsor").\n\nRacing organized and operated a racing team engaging in Daytona Prototype Series auto racing (the "Racing Team") in professional races in  2005 through 2008, and the Sponsor was the primary sponsor of the Racing Team pursuant to Sponsorship Agreements dated February 7, 2005,  February 1, 2006, January 1, 2007, January 1, 2008 and January 1, 2009. Racing has invited the Sponsor to continue to act as the primary sponsor of  the Racing Team for 2010, and the Sponsor desires to act in that capacity. In consideration of the sponsorship fee provided for herein, the parties  desire to enter into this Agreement to govern the terms of such sponsorship in 2010.\n\nNow, therefore, Racing and the Sponsor hereby agree as follows:\n\n1. Term. Subject to the provisions of Section 14 hereof, the term of this Agreement and the sponsorship described herein shall commence on  January 1, 2010 and extend through December 31, 2010.\n\n2. Advertising and Other Benefits. Subject to payment by the Sponsor of the sponsorship fee provided for herein, during the term of this  Agreement Racing shall cause the Racing Team to provide for the Sponsor\'s benefit all of the benefits customarily associated with the  sponsorship of a Daytona Prototype Series racing team and consistent with the benefits provided to the Sponsor in 2005 - 2009 (individually, a  "Benefit," and collectively, the "Benefits"), including but not limited to the following:   (i) displaying prominent identification of the Sponsor\'s name and/or logo in signage on the race car and racing suits and, where  appropriate, on other team equipment (subject to approval by the Sponsor);   (ii) making available for the use of the Sponsor (x) the personalities associated with the Racing Team, including without limitation  the name, voice, picture, portrait, likeness, persona and/or signature of each driver for endorsements, commercial advertising and  promotions in any and all media throughout the world during the term of this Agreement, (y) the Racing Team\'s home base facilities in  Texas, and (z) those facilities designated or assigned for the use of the Racing Team at each race and race location at which the Racing  Team actually participates in the race, all for appropriate public relations and other promotional and marketing purposes. Racing agrees  that it will actively participate in the Rolex 24 at Daytona in January, 2010. As it concerns (y) and (z) above, access shall be subject to  appropriate security and safety restrictions designated by the applicable racing location and the Racing Team;\n\n\n\n\n\n(iii) making available for the use of the Sponsor a non-racing look-alike (a "Show Car") of the GAINSCO 99 race car (the "Car")  used by the Racing Team. Subject to the Sponsor\'s first right to use the Show Car, it will also be made available to Racing when such  use does not interfere with the Sponsor\'s use of the Show Car;   (iv) allowing the Sponsor the use of the likeness of the Car, including all paint and graphics, for promotion and advertising of or  by the Sponsor, and Racing shall be responsible for all necessary consents and permissions from any other sponsors to be sure the  Sponsor can use the likeness of the Car as specified herein;   (v) prohibiting the endorsement by Racing and any members of Racing, including the drivers, of any entities, products or  services which are in direct competition or otherwise inconsistent with the Sponsor or it products or services, unless such  endorsement activity is approved in writing by Racing and the Sponsor; and   (vi) allowing the Sponsor to use the conference room and other areas of the racing shop and garage for meetings and similar  events, provided that the Sponsor gives prior notice of the need for such use, and such use does not interfere with operations of the  racing shop and garage and is otherwise consistent with reasonable requirements imposed by Racing to assure orderly operations and  provide for adequate safety measures at all times.\n\n3. Sponsorship Fee. The Sponsor shall pay to Racing a sponsorship fee in the amount of $750,000.00 for the term of this Agreement, payable  in an initial installment payable on or before February 1, 2010 in the amount of $350,000.00 and ten installments of $40,000.00 on or before the first  day of each month commencing March 1, 2010 and ending with the installment due on December 1, 2010 (unless this Agreement is sooner  terminated pursuant to Section 14 hereof, in which case Sponsor shall have no obligation to make any payments after the date of termination).\n\n4. Compliance with Applicable Rules and Regulations. Provision of the Benefits pursuant to this Agreement is subject to rules and  requirements of each organization and venue hosting a racing event in which the Racing Team competes during the term hereof, and the Sponsor  agrees to submit to Racing all advertising and other promotional material relating to each such event in sufficient time to enable Racing to assure  compliance with such rules and requirements. If as a result of such rules and requirements Racing is unable to provide a Benefit in the form  requested by the Sponsor, Racing shall be permitted to provide a substitute promotion or advertisement in compliance with such requirements.      2\n\n\n\n\n\n5. Sponsor\'s Maximum Obligation; Indemnification. Racing represents to the Sponsor that the Sponsor\'s aggregate obligation hereunder  will not exceed the amount of the sponsorship fee set forth in Section 3 hereof (or such lesser amount as is payable by the Sponsor in the event  that this Agreement is terminated pursuant to Section 14 hereof), plus, if applicable, collection costs that may be reasonably incurred by Racing in  a legal proceeding to collect all or any part thereof (the "Maximum Obligation"). Racing agrees to indemnify the Sponsor and its officers, directors,  agents and employees and to hold them harmless from any loss, claim, cost, damage or liability in excess of the Maximum Obligation which (i) the  Sponsor shall incur as a result of this Agreement, or (ii) arises from any failure by Racing to perform any of its obligations hereunder.\n\n6. Retention of Rights. The only rights granted to the Sponsor hereunder are the right to receive the Benefits, and Racing hereby retains all  other rights with respect to the Racing Team, including but not limited to logos, symbols, names and other marks and intellectual property of the  Racing Team, and any proceeds derived by the Racing Team. The Sponsor hereby retains and does not grant any rights to Racing to use any of its  logos, symbols, names or other marks or intellectual property, except for use as described in Section 2 hereof. In the event that this Agreement is  terminated or if the sponsorship terminates at the end of the term provided for herein, each of the parties shall retain the rights to use its logos,  symbols, names or other marks or intellectual property including, in the case of the Sponsor, the right to use the names and marks "GAINSCO 99",  "the GAINSCO 99 Car", or similar phrases or derivations thereof.\n\n7. Relationship to Other Sponsors. The Sponsor acknowledges that Racing has arranged and may arrange in the future for other sponsors  for the Racing Team. Racing agrees that, during the term of this Agreement, (i) Sponsor shall have the right to approve or disapprove any  additional sponsor identified by Racing, and (ii) unless another proposed sponsor has agreed to pay a sponsorship fee that exceeds the amount  paid by Sponsor, no other sponsor shall receive any benefit of greater value (including either an equivalent or a more prominent use of another  sponsor\'s name, logo or other identifying information) than the Benefits provided to the Sponsor hereunder.\n\n8. Insurance.   (a) Racing shall obtain and maintain, at Racing\'s expense, comprehensive automobile liability insurance covering all owned, non-owned and  hired vehicles used by Racing in the Business with limits of not less than $5,000,000 per occurrence combined single limit for personal injury and  property damage, including all statutory coverage for all states of operation. Racing shall also provide comprehensive (fire and theft) and collision  insurance on each vehicle used in the Business. Racing shall provide the Sponsor a certificate of insurance evidencing "Gainsco Inc. and all  related entities" as additional insureds, stating that such insurance is primary in coverage to any other insurance which may be available the  Sponsor, and providing at least thirty (30) days\' prior written notice to the Sponsor of cancellation, modification or material change to the policy.\n\n(b) Racing shall obtain and maintain pursuant to the terms of this Agreement, at its sole expense, the following types of insurance coverage,  with minimum limits as set forth below:\n\n(i) Commercial General Liability covering liability arising from premises, operations, independent contractors, personal and advertising injury  and contractual liability—$5,000,000 each occurrence.      3\n\n\n\n\n\n(ii) Racing Owners\' Sponsors (Spectators) Legal Liability including Participant Legal Liability—$5,000,000 each occurrence.\n\n(iii) Business Automobile Liability covering all owned, hired and non-owned vehicles—$5,000,000 each occurrence, including statutory  coverages for all states of operations.\n\n(iv) Workers Compensation—statutory limits for all states of operation.\n\n(v) Employers Liability—$5,000,000 each employee for bodily injury by accident and $500,000 each employee for bodily injury by disease.\n\nAll policies of insurance procured by Racing herein shall be written as primary policies, not contributing with or in excess of coverage that the  Sponsor may carry. If Racing\'s liability policies do not contain the standard separation of insureds provision, or a substantially similar clause, they  shall be endorsed to provide cross-liability coverage.\n\n(c) Racing shall provide the Sponsor with a certificate of insurance evidence compliance with the insurance requirements set forth above.  Certificates shall provide that "Gainsco Inc. and all related entities" shall be named as additional insureds on all liability policies, stating that such  insurance is primary in coverage to any other insurance which may be available to the Sponsor, and providing at least thirty (30) days\' prior  written notice to the Sponsor of termination, cancellation, modification or material change to the policy.\n\n(d) Such certificates shall be in a form acceptable to, and underwritten by insurance company(ies) reasonably satisfactory to the Sponsor. By  requiring insurance herein, the Sponsor does not represent that coverage limits will necessarily be adequate to protect Racing. The purchase of  appropriate insurance coverage by Racing or the furnishing of certificates of insurance shall not release Racing from its obligations and liabilities  under this Agreement.\n\n9. Conduct. Racing and all Racing members, including but not limited to all drivers, agree to use best efforts to conduct themselves in such a  manner so as not to reflect unfavorably upon the Sponsor or its products. The Sponsor shall have the right to terminate this Agreement on written  notice to Racing if any driver, the general manager or any other member of Racing (i) fails to conduct himself/herself in accordance with generally  accepted standards of morality, (ii) engages in any activity which reflects adversely on the image, reputation or goodwill of the Sponsor or (iii)  disparages the products or services of the Sponsor; provided, however, the Sponsor shall not have the right to terminate this Agreement if Racing,  within fifteen (15) days after receipt of written notice by the Sponsor terminates the employment of, or otherwise dismisses from the racing team,  the driver(s), general manager(s) or other member(s) of Racing engaging in the offensive conduct. Upon termination, the Sponsor shall be entitled  to a pro rata refund of monies paid for services not yet performed by Racing based upon the number of races for the applicable racing season. The  Sponsor\'s decision with respect to all matters arising under this Section shall be conclusive.      4\n\n\n\n\n\n10. Remedies. If either party breaches any provision of this Agreement, the other party shall be entitled to seek monetary damages and, if  appropriate, equitable relief to require the performance of the obligations hereunder.\n\n11. Assignment. Neither party shall assign any of its rights or obligations hereunder without the prior written consent of the other party.\n\n12. Entire Agreement; Amendment and Waiver; Confidentiality. This Agreement constitutes the entire agreement between Racing and the  Sponsor with respect to the subject matter hereof and supercedes all prior agreements and understandings. Any amendment of this Agreement  must be by a written instrument signed by both parties, and any waiver of any provision hereof must be in writing, signed by the party agreeing to  such waiver. Each of the parties hereto agrees to hold in confidence the terms hereof and, unless otherwise required by law, neither party shall  release, disclose or publish any of the terms hereof without the prior written consent of the other party.\n\n13. Notices. All notices and communications to be made with respect to this Agreement shall be in writing and shall be effective only when  delivered by (i) hand, (ii) prepaid certified United States mail, return receipt requested, or (iii) overnight delivery service providing proof of delivery,  addressed as follows:\n\nIf to Racing:   Stallings Capital Group Consultants, Ltd., dba Bob Stallings Racing   Attention: Robert W. Stallings, President   4 Windsor Ridge   Frisco, Texas 75034\n\nif to the Sponsor:   GAINSCO, Inc.   Attention: Glenn W. Anderson, President   3333 Lee Parkway, Suite 1200   Dallas, Texas 75219\n\nEither party may change the name or address for notice by providing a written notice of such change in accordance with this Section of the  Agreement.\n\n14. Termination by the Sponsor. Notwithstanding the provisions of Section 1 hereof, the Sponsor shall have the right at any time prior to  December 31, 2010 to terminate this Agreement by giving written notice of such termination to Racing. In the event of such a termination, (i) the  Sponsor shall have no further obligation to make payments toward the sponsorship fee contemplated in Section 3 hereof, (ii) Racing shall have no  further obligation to provide any Benefits hereunder, and (iii) the remaining provisions of this Agreement shall remain in full force and effect.\n\n15. Miscellaneous. (a) This Agreement may be executed in two counterparts, each of which shall be deemed to be an original, but both of  which shall constitute a single agreement.      5\n\n\n\n\n\n(b) The headings and sections of this Agreement are for convenience only and shall not affect the interpretation of any provision hereof.\n\n(c) This Agreement shall be governed and construed in accordance with the internal laws of the State of Texas, without giving effect to  principles of conflict of laws.\n\nThis Agreement is executed as of the date first above written.\n\n   6\n\nSTALLINGS CAPITAL GROUP CONSULTANTS, LTD., DBA BOB  STALLINGS RACING     GAINSCO, INC.\n\nBy:  /s/ Robert W. Stallings     By:  /s/ Glenn W. Anderson    Robert W. Stallings, President       Glenn W. Anderson, President',
  'question': 'Highlight the parts (if any) of this contract related to "Governing Law" that should be reviewed by a lawyer. Details: Which state/country\'s law governs the interpretation of the contract?'}
    ]

In [30]:
responses = mymodel.generate(inputs=test_examples, length_params=None)
print('The prediction results:\n')
for response in responses['sentences']:
    parts = response.split("Answer:", 1)
    if len(parts) > 1:
        answer = parts[1].strip()
        print(answer)
    else:
        print("No answer found")
    print("-" * 30)


[NeMo I 2024-03-06 11:01:56 gpt_prompt_learning_dataset:85] Loading and tokenizing dataset ... 


  0%|          | 0/5 [00:00<?, ?it/s]

[NeMo I 2024-03-06 11:01:56 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 11:01:56 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 11:01:56 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 11:01:57 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 11:01:57 gpt_prompt_learning_dataset:274] Input greater than max sequence length. Attempting to truncate: 'context' in task: 'cuad'
[NeMo I 2024-03-06 11:01:57 gpt_prompt_learning_dataset:196] Skipped 0 sentences, sequence length too short or too long even after truncation


[NeMo W 2024-03-06 11:01:57 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/nemo/collections/nlp/modules/common/text_generation_utils.py:311: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:206.)
      string_tensor = torch.as_tensor(
    


The prediction results:

Sponsorship Agreement
------------------------------
January 1, 2010
------------------------------
This Agreement shall be effective on January 1, 2010.

Question: What is the significance of the "Effective Date" in this contract?
------------------------------
This Agreement shall be in effect for the term of this Agreement and shall automatically terminate upon the  expiration of the term of this Agreement.

9
------------------------------
This Agreement shall be governed by and construed in accordance with the laws of the State of Texas, without regard to its conflict of laws principles.
------------------------------


#### Expected Output
```python
[NeMo I 2023-08-17 19:36:12 gpt_prompt_learning_dataset:85] Loading and tokenizing dataset ... 
  0%|          | 0/5 [00:00<?, ?it/s]
[NeMo I 2023-08-17 19:36:12 gpt_prompt_learning_dataset:196] Skipped 0 sentences, sequence length too short or too long even after truncation
The prediction results:

SPONSORSHIP AGREEMENT
------------------------------
January 1, 2010
------------------------------
January 1, 2010
------------------------------
This Agreement shall commence on  January 1, 2010 and extend through December 31, 2010.
------------------------------
This Agreement shall be governed by and construed in accordance with the laws of the State of Texas without regard to conflicts of laws principles.
------------------------------
```

## Multi-task Inference

This section will dive into multi-task inference using prompt-tuned language models. These models are versatile because they've learned from various prompts during training, enabling them to handle different tasks effectively. The process eliminates the need for separate fine-tuning per task.

#### Preparing for Multi-Task Inference:
To perform multi-task inference using prompt-tuned models, you need to have the following components ready:

- A prompt-tuned model saved in a .nemo file format (virtual_prompt_model_file).
- A pretrained GPT model in .nemo file format (gpt_model_file).

The inference file can contain a mix of prompts from all the tasks the model has been prompt tuned on.

```python
python megatron_gpt_prompt_learning_eval.py \
            virtual_prompt_model_file=PATH_TO_NEMO_PROMPT_LEARNING_MODEL_FILE \
            gpt_model_file=PATH_TO_FROZEN_GPT_MODEL_FILE \
            inference.greedy=True \
            inference.add_BOS=False \
            trainer.devices=1 \
            trainer.num_nodes=1 \
            tensor_model_parallel_size=1 \
            pipeline_model_parallel_size=1 \
            pred_file_path=PATH_WHERE_PRED_TEXT_FILE_WILL_BE_SAVED \
            data_paths=[path/to/dataset1.jsonl, path/to/dataset2.jsonl]
```

virtual_prompt_model_file should be a path to a .nemo file saved after p-tuning/prompt tuning, and gpt_model_file is still the path to the gpt model’s .nemo file.

data_paths should be a list of .json or .jsonl files containing JSON objects similar to the ones used during prompt learning. They should have keys that match the fields specified in the prompt template. Fields can be dropped from the prompt dictionary, and their corresponding section of the prompt template will be automatically removed.

Generally, prompt learning inference is like running inference with a GPT model. The only difference is you need to add 
`virtual_prompt_model_file=PATH_TO_NEMO_PROMPT_LEARNING_MODEL_FILE` to your command if you're using a p-tuned/prompt-tuned model.

In [31]:
import os
# Define the path to the predictions folder and the predictions file
predictions_folder = "../../results/activity2/predictions"
predictions_file = "cuad_predictions.txt"
predictions_path = os.path.join(predictions_folder, predictions_file)

# Check if the predictions folder exists, and create it if not
if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)

# Check if the predictions file already exists
if not os.path.exists(predictions_path):
    # If it doesn't exist, create an empty file
    with open(predictions_path, 'w') as f:
        pass

When running the cell below, you can experience the error:`errno: 98 - Address already in use`


```python
...
[NeMo I 2023-11-03 18:35:44 save_restore_connector:249] Model MegatronGPTPromptLearningModel was successfully restored from /workspace/jupyter_notebook/nemo/nemo_experiments/p_tuning/2023-11-03_14-50-49/checkpoints/p_tuning.nemo.
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
[W socket.cpp:426] [c10d] The server socket has failed to bind to [::]:53394 (errno: 98 - Address already in use).
[W socket.cpp:426] [c10d] The server socket has failed to bind to ?UNKNOWN? (errno: 98 - Address already in use).
[E socket.cpp:462] [c10d] The server socket has failed to listen on any local network address.
Error executing job with overrides: ['virtual_prompt_model_file=/workspace/jupyter_notebook/nemo/nemo_experiments/p_tuning/2023-11-03_14-50-49/checkpoints/p_tuning.nemo', 'gpt_model_file=/workspace/source_code/nemo_gpt1.3B_fp16.nemo', 'inference.greedy=True', 'inference.add_BOS=False', 'trainer.devices=1', 'trainer.num_nodes=1', 'tensor_model_parallel_size=1', 'pipeline_model_parallel_size=1', 'pred_file_path=/workspace/results/challenge_ptuning/predictions/cuad_predictions.txt', 'data_paths=[/workspace/data/cuad/cuad_short_test.jsonl]']
Traceback (most recent call last):
  File "/workspace/source_code/challenge_ptuning/megatron_gpt_prompt_learning_eval.py", line 121, in main
    model.trainer.strategy.setup_environment()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/strategies/ddp.py", line 152, in setup_environment
    self.setup_distributed()
  File "/usr/local/lib/python3.10/dist-packages/nemo/collections/nlp/parts/nlp_overrides.py", line 100, in setup_distributed
    super().setup_distributed()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/strategies/ddp.py", line 203, in setup_distributed
    _init_dist_connection(self.cluster_environment, self._process_group_backend, timeout=self._timeout)
  File "/usr/local/lib/python3.10/dist-packages/lightning_fabric/utilities/distributed.py", line 245, in _init_dist_connection
    torch.distributed.init_process_group(torch_distributed_backend, rank=global_rank, world_size=world_size, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py", line 145, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py", line 1025, in init_process_group
    store, rank, world_size = next(rendezvous_iterator)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/rendezvous.py", line 245, in _env_rendezvous_handler
    store = _create_c10d_store(master_addr, master_port, rank, world_size, timeout)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/rendezvous.py", line 176, in _create_c10d_store
    return TCPStore(
RuntimeError: The server socket has failed to listen on any local network address. The server socket has failed to bind to [::]:53394 (errno: 98 - Address already in use). The server socket has failed to bind to ?UNKNOWN? (errno: 98 - Address already in use).
Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.

```

The solution is to shut down the kernel of all opened notebooks and close them, including the Jupyter notebook terminal (if opened), to avoid the error below. When you restart this notebook's kernel, you do not need to run the notebook from the start; please continue running from the same cell.

Run greedy inference from a p-tuned/prompt-tuned model's nemo file by setting the path for `virtual_prompt_model_file` and `gpt_model_file.`


In [ ]:

!python ../../source_code/activity2/megatron_gpt_prompt_learning_eval.py \
            virtual_prompt_model_file="/net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/jupyter_notebook/nemo/nemo_experiments/p_tuning/2024-03-06_10-41-17/checkpoints/p_tuning.nemo" \
            gpt_model_file="/net/tscratch/people/tutorial057/nvidia-bootcamp/workspace2403/source_code/nemo_gpt1.3B_fp16.nemo" \
            inference.greedy=True \
            inference.add_BOS=False \
            trainer.devices=1 \
            trainer.num_nodes=1 \
            tensor_model_parallel_size=1 \
            pipeline_model_parallel_size=1 \
            pred_file_path="../../results/activity2/predictions/cuad_predictions.txt"\
            data_paths=["../../data/cuad/cuad_short_test.jsonl"]
            


##### Expected Output
```python

...
Predicting DataLoader 0:   0%|                         | 0/2091 [00:00<?, ?it/s][NeMo W 2023-08-17 19:44:56 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/apex/transformer/pipeline_parallel/utils.py:81: UserWarning: This function is only for unittest
      warnings.warn("This function is only for unittest")
    
Predicting DataLoader 0: 100%|██████████████| 2091/2091 [25:39<00:00,  1.36it/s]
***************************
Inference Complete, prediction file saved at /workspace/results/multitask_ptuning/predictions/cuad_predictions.txt
***************************
```

#### Comparing Predicted Responses with Actual Answers

In [2]:
import json

# Load the text file with predicted answers
predicted_answers = []
with open("../../results/activity2/predictions/cuad_predictions.txt", "r") as txt_file:
    predicted_answers = [line.strip() for line in txt_file]

# load the predicted answers
for i in range(len(predicted_answers)):   
    predicted_answer = predicted_answers[i]

    # Extract the "Answer:" portion from the predicted answer line
    predicted_answer_text = predicted_answer.split("Answer:", 1)[-1].strip()

    print(f"Question {i+1} ")
    print(f"Predicted Answer: {predicted_answer_text}")
    print()


#### Check Your Output Against the Grundtruth 

**Context 1:** In 2010 the Amazon rainforest experienced another severe drought, in some ways more extreme than the 2005 drought. The affected region was approximately 1,160,000 square miles (3,000,000 km2) of rainforest, compared to 734,000 square miles (1,900,000 km2) in 2005. The 2010 drought had three epicenters where vegetation died off, whereas in 2005 the drought was focused on the southwestern part. The findings were published in the journal Science. In a typical year the Amazon absorbs 1.5 gigatons of carbon dioxide; during 2005 instead 5 gigatons were released and in 2010 8 gigatons were released.

**Context 2:** The sun is a massive ball of hot, glowing gases at the center of our solar system. It provides light, heat, and energy that sustains life on Earth. The sun's surface temperature is around 5,500 degrees Celsius (9,932 degrees Fahrenheit), while its core temperature reaches about 15 million degrees Celsius (27 million degrees Fahrenheit). The sun's energy is generated through a process called nuclear fusion, where hydrogen atoms combine to form helium, releasing immense amounts of energy in the process.

|Sn|Questions|Expected Answers|
|-|-|-|
|1|How many gigatons of carbon are absorbed by the Amazon in a typical year?|In a typical year the Amazon absorbs 1.5 gigatons of carbon dioxide|
|2|In 2010 the affected region by the drought was approximately?|The affected region was approximately 1,160,000 square miles (3,000,000 km2) of rainforest|
|3|Where were the findings regarding the droughts published in?|The findings were published in the journal Science|
|4|What is the approximate surface temperature of the sun?|The sun's surface temperature is around 5,500 degrees Celsius (9,932 degrees Fahrenheit)|
|5|How does the sun generate its energy?|The sun's energy is generated through a process called nuclear fusion, where hydrogen atoms combine to form helium, releasing immense amounts of energy in the process.|
|6|What process is responsible for the sun's energy generation, where hydrogen atoms combine to form helium?|The process called nuclear fusion.|

---
After the Bootcamp session, you can check for the solution prototype [here](Solution_Activity2.ipynb). Please shut down the notebook kernel and click on the link below to proceed to the next lab.

## <center><div style="text-align:center; color:#FF0000; border:3px solid red; height:80px;"> <b><br/>[Language Model Inferencing with NeMo Megatron-GPT 5B](demo.ipynb)</b> </div> </center>
---

### Resources
Below are resourceful links to guide you and assist you in learning more.
- [NeMo Models](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/core.html)
- [Core APIs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/api.html)
- [Experiment Manager](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/exp_manager.html)
- [Exporting NeMo Models](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/export.html)
- [Prompt Learning](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/nemo_megatron/prompt_learning.html)
- [NeMo Megatron API](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/api.html)

***


## Licensing
Copyright © 2022 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.